# Supplementary Notebook for "Three-Dimensional Fault Morphology Obtained from Unsupervised Machine Learning of Clusters of Aftershocks." 

Authors: Brennan Brunsvik, Gabriele Morra, Gabriele Cambiotti, Lauro Chiaraluce, Raffaele Di Stefano,
Pasquale De Gori, David A. Yuen.

This Jupyter Notebook combines the paper and code to improve transparency
in our work and allow the reader to run their own tests. The code can be ran interactively in the cloud thanks to Binder.

This notebook was designed to be simple to use while still giving control. The majority of code has been placed into seperate files. Almost all code used for the paper is contained within the combination of these notebooks. The figures are present, though they may look different than in the main text. 

First, set up the notebook by importing everything and designating folders for use. 

In [ ]:
# THIS CELL CHOOSES HOW TO RUN THE NOTEBOOK. THE OTHER CELLS CAN BE LEFT ALONE.

runAllFigures = False


# The purpose of this code block is to decide whether to plot 3d interactive figures within the notebook
# or to save the plots as a file (temp-plot.html) that can be seen in the Supplementary_Jupyter_Notebook
# folder. The former option takes more memory and time, but automatically reloads. 
# The second option requires you to manually reload the file, but looks nicer. 
plotlyInNotebook = True
if plotlyInNotebook:
    from plotly.offline import iplot as plot
else:
    from plotly.offline import plot

In [ ]:
# For loading saved values. 
variable_folder = "stored_variables/"
slip_dist_folder = 'slip_dist/'

In [ ]:
#Import builtin
import os
import sys
import time
from datetime import datetime, date, time
import copy
import json

#sci and num py
import numpy as np
import scipy.optimize as optimize
from scipy.optimize import least_squares
from scipy.interpolate import Rbf
from scipy import interpolate
import scipy
import scipy.io as sio
from numpy.linalg import norm
from numpy.linalg import inv

import pandas as pd

#okada wrapper is crucial
try: 
    from okada_wrapper import dc3d0wrapper, dc3dwrapper 
except: 
    print('Please install the Okada Wrapper module.\
            https://github.com/tbenthompson/okada_wrapper') 

    
## visualization
# matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import matplotlib.patches as patches
from matplotlib.ticker import AutoMinorLocator
import matplotlib.mlab as mlab
import matplotlib
from mpl_toolkits.mplot3d import Axes3D
from scipy.spatial import ConvexHull
import matplotlib.path as mpltPath
from matplotlib.patches import FancyArrowPatch
from mpl_toolkits.mplot3d import proj3d

# Import plotly related stuff
#import chart_studio.plotly as py# replaced: import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode
init_notebook_mode(connected=True)
from scipy.spatial import Delaunay
import matplotlib.cm as cm
from functools import reduce

from IPython.display import Image
##

# For clustering
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import sklearn.mixture
import multiprocessing #!# may not need multiprocessing
import sklearn

# pyproj handles UTM projection
import pyproj

# import TDDispl
# import TDStrain

In [ ]:
# Functions for loading data. 
%run ./Load_Data.ipynb

In [ ]:
# Get functions that are used for many mathematical calculations. Open that file for details. 
%run ./Calculation_Functions.ipynb

In [ ]:
# Functions for fault splitting 
%run ./Fault_Splitting.ipynb #!# remove this

In [ ]:
# Get some fundamental functions for making figures. Open that file for details. 
%run ./Figure_Making_Functions.ipynb

In [ ]:
# Get clustering info
%run ./clustering_base_file.ipynb
print('Using old clustering file for now')

In [ ]:
# Get some stress related stuff
%run ./stress_base_file.ipynb 
# ! This shouldn't be needed, 01/04/2019

# Abstract

A lack of knowledge of the 3D structure of faults at depth results in several challenges. We propose a data-analytical method to unveil 3D rupture morphology of faults using unsupervised clustering techniques on earthquakes in seismic sequences. We apply this method to the 2009 L’Aquila sequence using about 50,000 relocated hypocenters. Using our 3D fault model, we invert slip distribution models using co-seismic displacement measured at 77 GPS stations and compare these results with a similarly inverted planar fault model as well as a high resolution planar joint inversion. We then use these models to find the induced changes in Coulomb stress (∆CFF) projected on 3415 aftershock focal mechanisms. We find that our slip model on a 3D surface, compared to the analogous planar model, finds fewer events with +∆CFF on focal mechanisms. However, it finds more events with +∆CFF when all aftershocks are used and assumed to be optimally oriented for failure. Ultimately, we find that ∆CFF is different between planar and 3D fault models only in proximity to the main fault. This new method shows promise as a step toward robustly and quickly obtaining 3D rupture morphologies where earthquake sequences have been monitored. Our technique also provides complimentary information to typical co-seismic data which will help to choose from non-unique, nonlinear mainshock inversion solutions.


# 1 Introduction


##  

The 3D morphology of faults and earthquakes is generally unknown at depth due to the lack of information which could constrain it. We present a new approach where we find the 3D morphology of faults and earthquake rupture surfaces based on the spatial distribution of aftershocks in seismic sequences. We apply this method to the 2009 L’Aquila seismic sequence in the central Apennines. Understanding 3D fault morphology can be useful in many ways. The new method gives insight into fault morphology, earthquake triggering, and can potentially be used to better assess seismic hazard.

We specifically apply our 3D fault model to study earthquake triggering. Understanding the earthquake source process and how earthquake sequences evolve requires accurate calculations of how stress changes due to an earthquake. Models of stress change induced by earthquakes can be performed using solutions that assume homogeneous slip (Brune, 1970; Okada, 1985, 1992). To mitigate the error from this assumption, faults are usually divided into many smaller patches with different slip (e.g. Harris and Segall, 1987; King, Stein, and Lin, 1994; Jiang et al., 2014). Due to the difficulty in knowing the 3D morphology of blind faults, planar fault geometry is usually assumed (for clarity, “fault plane” in this text does not refer to an active faults rupture surface and orientation. We use “plane” in a literal sense; for models of faults and/or rupture which have no curvature. This contrasts to the 3D morphology of faults and/or rupture, which can have a non-planar shape). 

Planar mainshock models may be problematic for modeling stress transfer. Static stress transfer and its earthquake implications are not fully understood. Mainshock originating ∆CFF (a measurement of how a mainshock promotes failure on faults) on aftershocks is often only slightly biased towards positive, suggesting that static stress only partially explains aftershocks. Because mainshock fault morphology is usually modelled as planar, the capability for fault morphology inaccuracy to interfere with the results should be considered, and methods to reduce this inaccuracy should be developed. We use our 3D rupture morphology approach, combined with GPS based co-seismic slip distribution inversion, to address this problem in the L’Aquila sequence. 

Chiaraluce (2012) provided a review of the seismic activity in the central-northern Apennines with a focus on the 2009 L’Aquila sequence (Figure 1). The L’Aquila sequence activated a south-west dipping normal fault system about 50 km long. Paleo-seismological trenching of the Paganica fault zone, considered to be the geological source responsible for the L’Aquila main shock with MW 6.1 (Scognamiglio et al., 2010), suggests a surface rupture recurrence interval between 700 and 1250 years with the possibility of previous earthquakes with MW>6.5 (Cinti et al., 2011), larger than the 2009 mainshock. The L’Aquila sequence is part of a larger extensional fault system that has experienced several recent earthquakes. The 1979 Umbria sequence with a MS 5.9 mainshock (Deschamps, Iannaccone, and Scarpa, 1984) occurred between the 2009 L’Aquila and 1997 Colfiorito earthquake sequences (Chiaraluce et al., 2003). The 1997 Colfiorito sequence with a MW 6.0 mainshock occurred approximately 60 km north-northwest of the L’Aquila sequence and had several 5<MW<6 earthquakes (Chiaraluce et al., 2003; Nostro et al., 2005). In October 2016, a seismic sequence occurred near Amatrice, northwest of the L'Aquila seismic sequence, with a maximum magnitude of 6.5 (Chiaraluce et al., 2017). 

The combination of nearby seismic sequences shows rupturing on related fault systems extending for more than 150 km. These sequences are interconnected in that each modifies the stress throughout the fault system, promoting future seismic sequences in some areas and inhibiting them in other areas. Verdecchia et al. (2018) showed that ∆CFF and viscoelastic stress transfer have linked several of these sequences together. This is true in the short and long term. They showed that the L’Aquila sequence experienced ~1 bar of ∆CFF beginning with the 1915 MW 6.9 Fucino earthquake up until rupture in 2009. This suggests the importance of modelling stress transfer between sequences as accurately as possible for scientific and hazard assessment purposes. 

Modification of fluid properties can result in rupture and is potentially an important component of seismic sequences. If fluid pressure is increased, it counteracts the normal stress that keeps a fault stable. This can happen in the dynamic sense due to passing seismic waves (Hill et al., 1993), or due to static stress transfer (Cocco and Rice, 2002). It is also possible for high pressure fluids to migrate into fault zones and cause seismicity. The 1997 Umbria-Marche sequence in the northern Apennines was proposed to be controlled by high-pressure CO2 entering and releasing faults (Miller et al., 2004). Fluids have also been proposed to be very important throughout the L’Aquila seismic sequence (Di Luccio et al., 2010; Lucente et al., 2010; Malagnini et al., 2012). This is supported in part based on temporal changes in vp/vs ratios, and northward migration of aftershocks at rates comparable to that of fluid flow. 

Earthquakes associated with the L’Aquila sequence started at the end of 2008. Events continued for about 4 months and included a MW 4.0 earthquake on March 30th before the mainshock ruptured on April 6th (Chiaraluce et al., 2011). Before the mainshock, events were focused along ~10 km of the fault segment in the deepest part of the L’Aquila fault. These have been extensively studied and described (e.g. Di Luccio et al., 2010; Lucente et al., 2010; Chiaraluce, 2012; Valoroso et al., 2013). The aftershocks occurred on three primary SE striking and SW dipping fault systems: L’Aquila, Campotosto, and Cittareale (Figures 1 and 2). The mainshock and a large portion of aftershocks occurred on the L’Aquila fault. The Cittareale fault is NW of the L’Aquila fault, and the Campotosto fault is shifted east from the L’Aquila and Cittareale faults. The aftershocks included several MW>5 events (Chiaraluce, 2012). Southeast of the mainshock, a MW 5.4 event occurred on the L’Aquila fault. Campotosto fault seismicity included a MW 5.0 event on April 6th, and MW 5.0 and 5.2 events on April 9th. These events occurred on the shallower, steeply dipping portion of the Campotosto fault. The deeper, steeply dipping section of the Campotosto fault appears kinked at its base where seismicity abruptly changes to low dip. This kink may have partially controlled the development of aftershocks; a MW 4.4 event occurred later on June 22nd beneath the kink, on the gently dipping portion of the Campotosto fault. On the 25th of June, a 3.9 MW earthquake occurred on a normal fault near Cittareale toward the northernmost portion of the L’Aquila seismic sequence. 

#### Figure 1
Figure 1. Modified from Chiaraluce et al. (2017). Map view of local seismicity in the central Apennines. The inset map shows specifically the L’Aquila sequence that we study. Dark blue crosses show the 2009 L’Aquila seismic sequence. Light blue crosses show the 1997 Colfiorito seismic sequence. Black dots show the Amatrice-Visso-Norcia sequence which started in 2016. Black squares show historical earthquakes. Green stars are events with MW≥ 5.0. Red stars are events with MW ≥ 6.0. Focal mechanisms are for events with MW≥ 5.0. Yellow stars and associated focal mechanisms are for events on January 18, 2017 with 5.0≤ML≤5.5.

In [ ]:
Image(filename='Figure_Map_corrected.jpg') 

#### Figure 2
Figure 2. This is a simplified schematic of the fault system involved in the L’Aquila sequence. Seismic activity occurred dominantly on the L’Aquila, Campotosto, and Cittareale faults. The L’Aquila fault hosted the mainshock. The faults strike approximately SE.

In [ ]:
matplotlib.rcParams.update({'font.size': 16})


# approximate fault locations, sizes, dips
p1 = np.array([11.15, -15.17, -7.2])
p2 = np.array([-10.58, 10.88, -8.98])
p = p1-p2
laqstrikewidth = np.linalg.norm(p)
laqdipwidth = .4 * laqstrikewidth

camstrikewidth = 2/3 * laqstrikewidth
camdipwidth = .4 * camstrikewidth 

citstrikewidth = np.array([10])
citdipwidth = .4 * citstrikewidth

cit = [-14.8,  25.5, -5.09, citstrikewidth, citdipwidth, - 5 * np.pi/4, 50 * np.pi/180]
cam = [-2.83,  17.3, -8.55, camstrikewidth, camdipwidth, - 5 * np.pi/4, 50 * np.pi/180]
laq = [ 4.31, -3.64, -6.78, laqstrikewidth, laqdipwidth, - 5 * np.pi/4, 50 * np.pi/180]
allFaults = [cit, cam, laq]

def makeSchemSurfPoints(x, y, z, xWidth, yWidth, s, d, reps = 2):
    # return points on at outside of fault based on inner point of fault
    __, strikeUnit = faultUnitVectors(s, d, 0       )
    __, dipUnit    = faultUnitVectors(s, d, -np.pi/2)
    
    points = np.zeros((reps, reps, 3))
    points[:, :] = np.array([x, y, z])
    
#     mults = np.array([-.5, .5])
    mults = np.linspace(-.5, .5, reps)
    for i, m1 in enumerate(mults):
        for j, m2 in enumerate(mults):
            points[i, j] += m1 * strikeUnit * xWidth + m2 * dipUnit * yWidth
            
    xs = points[:, :, 0].ravel()
    ys = points[:, :, 1].ravel()
    zs = points[:, :, 2].ravel()
    
    return xs, ys, zs




fig=plt.figure(figsize=(12, 12))
ax=fig.add_subplot(111,projection='3d')

for fault in allFaults:
    reps = 2
    x, y, z = makeSchemSurfPoints(*fault, reps = reps)
    x=x.reshape((reps, reps))
    y=y.reshape((reps, reps))
    z=z.reshape((reps, reps))
    ax.plot_surface(x, y, z, linewidth=1, edgecolors='k', alpha = .7)
   
arrow = [np.array([-20, -20]),
         np.array([0, 10]),
         np.array([0, 0]) ]
class Arrow3D(FancyArrowPatch):
    # arrows in 3D
    def __init__(self, xs, ys, zs, *args, **kwargs):
        FancyArrowPatch.__init__(self, (0,0), (0,0), *args, **kwargs)
        self._verts3d = xs, ys, zs

    def draw(self, renderer):
        xs3d, ys3d, zs3d = self._verts3d
        xs, ys, zs = proj3d.proj_transform(xs3d, ys3d, zs3d, renderer.M)
        self.set_positions((xs[0],ys[0]),(xs[1],ys[1]))
        FancyArrowPatch.draw(self, renderer)
a = Arrow3D(*arrow, mutation_scale=20, 
                lw=3, arrowstyle="-|>")
ax.add_artist(a)
ax.text3D(arrow[0][1], arrow[1][1], arrow[2][1], 'N')

arrowUp = [arrow[0],
         np.array([0, 0]),
         np.array([0, 10])]
a = Arrow3D(*arrowUp, mutation_scale=20, 
                lw=3, arrowstyle="-|>")
ax.add_artist(a)
ax.text3D(arrowUp[0][1], arrowUp[1][1], arrowUp[2][1], 'UP')


scaleBar = [np.array([-15,  -15]),
            np.array([0 ,  10]),
            np.array([0, 0]) ]
b = Arrow3D(*scaleBar, mutation_scale=20, lw=3, arrowstyle = '-')
ax.add_artist(b)
ax.text3D(scaleBar[0][1]+1, scaleBar[1][1] - 5, scaleBar[2][1], '10 Km')

labels = ['Cittareale', 'Campotosto', 'L\'Aquila']
for i, fault in enumerate(allFaults):
    ax.text3D(fault[0], fault[1], fault[2], labels[i])
    
l = ax.get_xlim()
ax.set_ylim(l)
ax.set_zlim(l)

groundrange = np.array(l, l)
xg, yg = np.meshgrid(groundrange,groundrange)
zg = np.zeros(xg.shape)
ax.plot_surface(xg, yg, zg, alpha = .1)

ax.set_xlabel('East (Km)')
ax.set_ylabel('North (Km)')
ax.set_zlabel('Vertical (Km)')


ax.xaxis.pane.set_edgecolor('black')
ax.yaxis.pane.set_edgecolor('black')
ax.zaxis.pane.set_edgecolor('black')
ax.grid(False)

# ax.view_init(elev = 20, azim = -10)

plt.show()

## 1.1 Rupture Morphology and Clustering of Earthquakes

At a large scale, for instance through the Wadati Benioff Zone, the 3D morphology of subducting slabs plates has been determined (Gudmundsson and Sambridge, 1998; Syracuse and Abers, 2006; Hayes, Wald, and Johnson, 2012). For megathrusts, for instance the 7.8 MW 2015 Gorkha earthquake, fault morphology (Hubbard et al., 2016) has been recently incorporated into stress studies (Qiu et al., 2016; Landry and Barbot, 2018) providing important information on locked zones (Avouac, 2008; Avouac et al., 2015). Locked zones are crucial to understand as they can be a source of intense seismicity. For the major subduction zones of the Earth, it has been shown that curvature on a fault is an important control for earthquake rupture area and magnitude (Bletery et al., 2016), suggesting that understanding fault morphology is important for assessing seismic hazard. For strike-slip faults, 3D fault morphology is sometimes estimated by projecting mapped surface ruptures into the ground. For moderate sized dip-slip earthquakes, 3D morphology is more difficult to obtain.

The lack of 3D fault model detail is problematic as it is not always reasonable to assume that a fault lies on a plane. This is shown in field work (e.g. Cinti et al., 2011) and seismic imaging (e.g. Lohr et al., 2008). For example, in extensional systems, normal faults may change their morphology dramatically along strike or with depth. An example of this is given within our study area by the Campotosto fault showing an abrupt change in dip toward depth from high angle to low angle (Chiaraluce, 2012). However, rupture can also occur on several smaller faults that are not necessarily coplanar but instead link together through complex fractures (Kim, Peacock, and Sanderson, 2004; Childs et al., 2009). Further, it is known that 3D fault roughness is an important component regarding earthquake magnitude and stress generation (Zielke, Galis, and Mai, 2017). Inaccuracies in fault morphology can strongly interfere with earthquake source inversions resulting in unrealistic slip distributions (Ragon, Sladen, and Simons, 2018) and unrealistic ∆CFF calculations. This illustrates the importance of developing and improving methods to retrieve and incorporate accurate 3D morphology of faults in stress calculations. 

With 3D seismic imaging, understanding deep fault morphology is not a new task. Because fault damage zones have low seismic velocity, they can create head waves and trapped waves in response to passing seismic waves. Modelling of such waves gives a way to see fault structures (Ben-Zion et al., 2003). Sorting through active source image data to locate points lying on faults can allow software guided interpolation to reconstruct fault surfaces (e.g. Lohr et al., 2008; Røe, Georgsen, and Abrahamsen, 2014). For instance, the 3D morphology of the low angle Altotiberina normal fault in northern Italy, obtained from merging seismic imaging, borehole analysis, geologic maps, and field surveys, was used to model stresses associated with creep and to estimate a deterministic velocity model (Mirabella et al., 2011 and references therein; Anderlini et al., 2016; Latorre et al., 2017). Thanks to a recent triangular fault patch slip solution (Nikkhoo and Walter, 2015), instead of the more traditional Okada (1985, 1992) model which uses rectangular fault patches, there have been numerous stress experiments with 3D morphologies (e.g. Qiu et al., 2016; Landry and Barbot, 2018). Dynamic stress simulations have also incorporated 3D morphologies (e.g. Pelties et al., 2012; Galvez et al., 2014; Zhang, Zhang, and Chen, 2014). 

Unfortunately, the costs of seismic imaging make this a difficult option for finding the shape of relatively small active faults. This suggests the need for different methods for finding the morphology of a mainshock. The 2009 L’Aquila earthquake sequence provides an excellent opportunity to reconstruct the fault morphology and also to apply slip inversion methods to test the importance of incorporating the fault curvature in stress calculations. This is because the seismic sequence was particularly well monitored, and a great deal of data including hypocenter locations, focal mechanisms, and GPS data are available. With this large amount of data, we attempt to reconstruct the complex curvature of the fault that hosted the mainshock by interpolating for the best fit fault surface within clusters of aftershocks. With the fault morphology reconstructed, we attempt to gain further insight into earthquake interaction by calculating ∆CFF induced by this 3D model as well as planar mainshock models. 

We employ unsupervised machine learning clustering for our methods (Pedregosa et al., 2011). Clusters of similar data (e.g. location, time of occurrence, magnitude, etc.) can be found using these techniques. See Kong et al. (2018) and Bergen et al. (2019) for an overview of machine learning in seismology and Earth sciences. Clustering of earthquakes has been used extensively for hypocenter relocation. Initial estimates of hypocenter locations are generally poor, but several techniques can improve the accuracy of these already located earthquakes. In the collapsing method (Jones and Stewart, 1997), the centroid of a cluster of nearby events is used to relocate hypocenters by shifting them toward the position of the cluster’s centroid. Many relocation techniques take advantage of the correlation in travel path of the waves originating from clustered earthquakes. Jordan and Sverdrup (1981) developed the hypocentroid decomposition theorem, which has been used in other cluster based relocation techniques (e.g. Bergman and Solomon, 1990; Karasözen et al., 2016). Cross-correlation of waves for similar clustered events has been used for relocation (e.g. Poupinet, Ellsworth, and Frechet, 1984; Dodge, Beroza, and Ellsworth, 1995; Shearer, 1997). For the L’Aquila sequence, the double difference relocation method (Waldhauser and Ellsworth, 2000) was used for the relocation of aftershocks (Chiaraluce et al., 2011), and extension of the aftershock catalog (Valoroso et al., 2013). Generally, machine learning methods of clustering are not relevant to these mentioned studies. Recently, Hierarchical clustering, a machine learning algorithm, was used in GrowClust (Trugman and Shearer, 2017) for earthquake relocation. Clustering has also been used on various earthquake features to investigate earthquake processes and precursors (Dzwinel et al., 2003; Dzwinel et al., 2005; Yuen et al., 2009). 

Spatial clustering of earthquakes with applications outside of hypocenter relocation have been done as well. Leśniak and Isakow (2009) applied clustering to assessing seismic safety in mines. After locating earthquakes, they clustered the hypocenters using several algorithms based on location and time of occurrence. They then assessed seismic hazard associated with the most dangerous clusters. Unlike our methods, they did not attempt to cluster with the assumption that earthquakes lie on the same faults because this was not relevant to their study. Rietbrock et al. (1996) used waveform clustering to find about 20 different clusters of earthquakes in the Gulf of Corinth. The purpose was to remove ambiguity regarding certain focal mechanisms to determine if low angle normal faulting had been observed. They found an RMS least-error plane to fit the cluster of earthquakes and determined that these events did display low angle normal faulting. However, fault morphology detail was not part of the study. Moment tensor based clustering has also been used (Cesca, Şen, and Dahm, 2013).


## 1.2 Summary of Methods

The L’Aquila earthquake sequence was particularly well monitored, allowing for relocation of over 50,000 aftershocks throughout 2009 (Valoroso et al., 2013). Using the aftershock locations, we cluster them into different faults. This allows us to interpolate for a fault surface that best fits a clustered set of hypocenters. We distribute small segments of fault cells oriented according to the interpolated surface and we calculate stress as the sum of stresses generated by these cells. GPS coverage of the mainshock was exceptional with 77 3-component co-seismic displacements available (Serpelloni et al., 2012). We use this to invert for and compare the best fit slip distribution for planar faults and for the 3D fault. We calculate ∆CFF that resulted from these fault models onto ~3400 aftershocks. We compare our results to the planar joint inversion of Cirella et al. (2012). The Python code is written in Jupyter Notebooks (Oliphant, 2006; Kluyver et al., 2016; Morra, 2018). It is available in GitHub (https://github.com/brennanbrunsvik/Fault-morphology-clustering) where it can be ran through the cloud service Binder. 

# 2 Clustering

## 2.1 Methods

The availability of such a large dataset of L’Aquila sequence earthquake locations allows for high precision analysis of the anatomy of the fault system (Valoroso et al., 2013). In order to interpolate a fault surface based on hypocenters, we first associated aftershocks with their respective faults. To distinguish between different faults in the hypocenter data, we perform clustering on the hypocenters based on their spatial locations. This presents many challenges. Where faults intersect, hypocenters of different faults can be very close to each other and clustering algorithms can be blind to this. Also, aftershock location inaccuracy is inevitably a problem due to imperfect knowledge of crustal 3D velocities and elastic properties. Thus, hypocenters tend to erroneously appear between multiple faults, and which fault these hypocenters belong to is unclear. To mitigate this, clustering is done in two iterations; first with a spectral method, and second with the DBSCAN method. Many overviews of clustering algorithms are available (e.g. Omran, Engelbrecht, and Salman, 2007; Joshi and Kaur, 2013).

During the first clustering iteration, we used the spectral clustering algorithm from Scikit-Learn (Pedregosa et al., 2011). Spectral clustering is good at distinguishing clusters when data are connected but a point’s distance from the center of the cluster is irrelevant, and when the clusters are non-convex. This algorithm computes the Gaussian affinities (the quantitative similarity between points) based on the distances between points. It then creates a normalized Laplacian matrix and clusters based on the eigenvectors. See Von Luxburg (2007) for more details on spectral clustering. We used only the 343 earthquakes with ML >= 2.5 during the first clustering attempt. This helps to remove small aftershocks that blur the distinction between different faults. It also makes the memory demanding spectral clustering algorithm more stable and manageable. This step separated the hypocenters into three dominant clusters. 

The second clustering iteration requires a modified data set to enhance fault like trends. For the cluster containing the mainshock, we found a least squares distance best fit plane to the hypocenters. The positions of all hypocenters were then stretched by a factor of 5 in the direction normal to the best fit plane. The stretching factor is subjectively chosen for ideal performance on the L’Aquila fault. The purpose is that trends along the fault surface are left alone, but deviations from this surface are amplified. With planar like trends more pronounced, clustering algorithms are much more successful at clustering long faults and keeping them separate from neighboring faults. This also reduces the problem that some hypocenter groups may lie along the same fault, but there are so few hypocenters between these groups that clustering algorithms normally find them as separate clusters. This spatial modification allows for the use of less computationally expensive clustering methods. 

For the second clustering iteration on the modified hypocenters, we used DBSCAN (Density-Based Spatial Clustering of Applications with Noise; Ester et al., 1996), which is available through Scikit-Learn (Pedregosa et al., 2011). This works by first finding a sample that has a minimum number of other samples, MinPts, within a minimum distance, Eps, and establishes this as a core sample of a cluster. Nearby core points become part of the same clusters. Points that do not satisfy the MinPts condition but are near another core point are considered part of the same cluster as the core point, but they are not themselves core points. Points that (i) have distance > Eps from all core points, and (ii) cannot become the core point for a new cluster are considered noise and are excluded from all clusters. We chose Eps=2.0 km and MinPts=80 samples. The selection of these values is somewhat arbitrary and choosing values that work well in one location may not work well in other locations. MinDist and Eps were chosen to perform best for the L’Aquila cluster with some expense to performance of the other clusters.



In [ ]:
# object with info for all hypocenters. 
allHypo = hypoData(minMag=None)

allHypo.labels=np.zeros(allHypo.mL.size)

if False:
    allHypo.plot3dclusters(pointSize=1, save=False)

# Spectral clustering
sp = spClust()
sp.minMag=2.5
sp.getHypocenters()

sp.assign_labels="discretize"
sp.n_init=10
sp.beta=1
sp.n_clusters=3 # choose this before hand based on the 3 main faults

sp.cluster()
# spectral clustering is completed 
# sp.labels indicates cluster identities
if False:
    sp.plot3dclusters(pointSize=7)

In [ ]:
# Make a best fit plane to the cluster containing the mainshock

laqFault = sp.labels==2 # make bool for hypocenters in the L'Aquila fault cluster

x = sp.x[laqFault]
y = sp.y[laqFault]
z = sp.z[laqFault]
mL = sp.mL[laqFault]
strikeHyp = sp.st1[laqFault]
dipHyp = sp.dp1[laqFault]
strike2Hyp = sp.st2[laqFault]
dip2Hyp = sp.dp2[laqFault]

focalBool = (~( (sp.st1==0) * (sp.st2==0) * (sp.dp1==0)
               * (sp.dp2==0) * (sp.rk1==0) * (sp.rk2==0) ))[laqFault]

# maxMag = 4
# magBool = (mL<=4)


splineNodesLine = 7

# exclude points over 10km from plane. Do rms fit. 
laqPlane = interp(x, y, z, 
               strikeHyp=strikeHyp, dipHyp=dipHyp, strike2Hyp=strike2Hyp, dip2Hyp=dip2Hyp,focalBool=focalBool,
               eps=.1, magnitudeWeight=mL, minPlaneDist = 10000,exp=2,
               splineNodesStrike = splineNodesLine, splineNodesDip = splineNodesLine)

# Prepare for second clustering attempt by "stretching" hypocenters.
db2 = dbClust() #!# rename db2 to db
db2.minMag=None # use all earthquakes
db2.getHypocenters()
db2.geogToMet()

# p is vector of points.
db2.p = rotateUniVec(db2.p-np.array([laqPlane.xC,laqPlane.yC, laqPlane.zC]),
    laqPlane.strikeHat, laqPlane.dipHat, laqPlane.normalHat)

# stretch points away from plane, but not along plane. 
db2.p[:,2]*=5

# distances from plane in new coordinate system
distEach = np.abs(distFromPlane(db2.x, db2.y, db2.z, 
                                laqPlane.xC, laqPlane.yC, laqPlane.zC, 
                                laqPlane.normalHat[0], laqPlane.normalHat[1], laqPlane.normalHat[2]))

# We get a warning regarding matrix deficiency, but this is not relevant to the aquired results.

In [ ]:
db2.eps=2.0* 1e3
db2.min_samples=80
db2.fitdbscan(distEach) # cluster in modified axis system

#### Figure 3
Figure 3. Also available interactively online at https://plot.ly/~BrennanBrunsvik/16/. This shows the final results from clustering hypocenters from the L’Aquila seismic sequence. Spectral clustering was first used to find three primary clusters. DBSCAN was then used on all hypocenter location data. The darkest blue dots correspond to noise. Other dark blue dots correspond to the Paganica fault, light blue to the Campotosto fault, and red to the Cittareale fault. Other clusters were found which do not necessarily relate to known faults.

In [ ]:
if False or runAllFigures:
    db2.plot3dclusters(pointSize=1.6, save = False, lineWidthR=1e-10)
    
# if True:
#     db2.plot3dclusters(pointSize=.8, save = True, lineWidthR=1e-10, name = 'Figure S3')

## 2.2 Results

The final clusters, the results of DBSCAN, can be seen in Figure 2. The reader is also referred to an online interactive 3D plot (Figure S1). Seven clusters were recognized of which some can be visually identified as major faults. The separation between major faults and scattered single aftershocks (referred to as noise and rejected by the DBSCAN algorithm) appears successful. However, sets of faults antithetic to and intersecting the mainshock cluster contained a high density of aftershocks, and these were not distinguishable from the main fault through clustering (Figure 4b). Clustering also failed to separate the Campotosto fault from a synthetic fault lying between the L’Aquila and Campotosto faults. This, however, did not affect the mainshocks cluster, which is the main topic of this study.  

# 3 Fault Surface Construction

## 3.1 Methods

The hypocenters in the clustered fault which contained the mainshock were spatially interpolated to make a best fit 3D fault surface. The following coordinate system is used: $x_1’$ faces along dip to the southeast, $x_2’$ faces up dip, and $x_3’$ faces the hanging wall, normal to the plane. The bi-cubic splines of Scientific Python (Jones et al., 2014) provide an excellent tool to interpolate the hypocenters. On the plane that best fits the aftershock locations of the L’Aquila fault cluster, we created a 7x7 grid of knots (nodes on which spline parameters are solved) extending from the minimum to the maximum hypocenter locations in $x_1’$ and $x_2’$ such that all of the cluster is contained within the knot grid. Knot parameters are inverted so that the surface provides a least square error distance to the hypocenters.

In [ ]:
# Create boolean showing which quakes are in L'Aquila cluster

# Then create boolean of those for which focal mechanisms are available
booLaqdb = db2.labels==0

allPoints = hypoData(minMag=None)
allPoints.getHypocenters()

x2 = allPoints.x[booLaqdb]
y2 = allPoints.y[booLaqdb]
z2 = allPoints.z[booLaqdb]
st1 = allPoints.st1[booLaqdb]
st2 = allPoints.st2[booLaqdb]
dp1 = allPoints.dp1[booLaqdb]
dp2 = allPoints.dp2[booLaqdb]
focalbooLaqdb = (~( (allPoints.st1==0 ) * (allPoints.st2==0 ) * (allPoints.dp1==0 )
  * (allPoints.dp2==0 ) * (allPoints.rk1==0 ) * (allPoints.rk2==0 ) ))[booLaqdb]

In [ ]:
# # changing on 02012020
# def rotateUniVec(p, e1, e2, e3):
#     """returns pPrime
#     p is array of vectors in original coordinates of shape (samples, 3)
#     e1, e2, and e3 are the new unit vectors in the old axis, each of shape (3)"""
#     points = p[:,0].size
    
#     x = np.zeros(points)
#     y = np.zeros(points)
#     z = np.zeros(points)

#     for i in np.arange(points):
#         x[i] = np.dot( p[i], e1 ) # !!
#         y[i] = np.dot( p[i], e2 ) # !! 
#         z[i] = np.dot( p[i], e3 ) 
        
#     pPrime = np.array([x, y, z]).T
    
#     print(x[:3])
#     print(y[:3])
#     return
        
#     return pPrime

In [ ]:
# surface.spline.get_knots()[1].shape

In [ ]:
# %%capture 
# get rid of a unimportant warning

splineNodesLine = 7 # changing from 7, 02/15/2020
cutPoints = True # convex hull. 02/15/2020

# note that surface is object for spline interpolation
# surf is object for our 3D mainshock model
surface = interp(x2, y2, z2,
               strikeHyp=st1, dipHyp=dp1, strike2Hyp=st2, dip2Hyp=dp2, focalBool=focalBool,
               eps=.01, minPlaneDist = 10000,exp=2,
               splineNodesStrike = splineNodesLine, splineNodesDip = splineNodesLine, 
               cutPoints = cutPoints, smooth = 1e4)
# eps = .01, smooth = ??




surface.focalAvailable=focalbooLaqdb

# if True:
#     print('this is a test! remove it!')
#     surface.pS *= -1
#     surface.pD *= -1 
#     print("It doesn't work!")

# plotSurface(surface)

In [ ]:
### testing problems on 02032020
# different: surface.pS
# approximately negative? : surface.e1Prime
# approximately negative? : surface.pPrime
# slightly different... surface.xC Maybe just some different points were included. 
# negative: surface.strikeHat
# negative: surface.dipHat
# Same!: surface.normalHat
# StrikeHat and dipHat are different. strikeHat faces the non right hand rule direction. 
# Where was strikeHat made? 

# even if I make strikeHat and dipHat negative, figure 5 gets close (it isn't flipped)
# BUT, there is a new huge bump in the fault and there is some new aftershocks that weren't there before.

# There are the same number of earthquakes in both notebooks. 
# However, the fault shape was made from 29357 earthquakes and now it's 29993

# I thus thing the clustering has changed...

# Somehow, the spectral clustering now finds one more earthquake. 

# the laqPlane is much different... the center points are about 10% changed. 

# !!! the dip of laqPlane is now only 25 degrees, it was previously 45!

In [ ]:
# Here, I save info from surface object to upload into a mainshock model.
# It has to be ran now, but is not relevant until the slip distribution inversion.

surface.gridNodesStrike = 40
surface.gridNodesDip = 24

#the outermost cell center be inset from the outermost part of surface:
shiftS = (np.amax(surface.pS) - np.amin(surface.pS)) / surface.gridNodesStrike * .5
shiftD = (np.amax(surface.pD) - np.amin(surface.pD)) / surface.gridNodesDip * .5
# Generate points on surface
surface.splineInterp(cutPoints=False, sEdgeShift = shiftS, dEdgeShift = shiftD)

# lat lon and depth of okada cells
latSurf, lonSurf = allPoints.metToGeog(surface.interpX, surface.interpY)
depthSurf = surface.interpZ

# Surface geometry (strike dip, etc.)
surface.surfaceNormal(surface.interpS, surface.interpD)
stSurf, dpSurf = normalToStrikeDip(surface.normalAprox)
surface.simpleCellWidths()
strikeWidth = surface.strikeWidth
dipWidth = surface.dipWidth

# 2d position for spline interpolation
interpS = surface.interpS
interpD = surface.interpD

# I save this as an array and load it later into a different class.
saveArrays([latSurf, lonSurf, depthSurf, stSurf, dpSurf, interpS, interpD, strikeWidth, dipWidth],
          fName = 'surfaceArraysCoarse', path = variable_folder)
##########

In [ ]:
###########
# TRIANGULAR DISLOCATION VERSION
# this is the same as above, but for the triangular dislocation.
# the difference is that these points cover the entire surface,
# while the rectangle points do not touch the edge of the surface
# the variables that I save are different as well

surface.gridNodesStrike = 40
surface.gridNodesDip = 24

#the outermost cell center be inset from the outermost part of surface:
shiftS = 0
shiftD = 0 
# Generate points on surface
surface.splineInterp(cutPoints=False, sEdgeShift = shiftS, dEdgeShift = shiftD)

# lat lon and depth of okada cells
latSurf, lonSurf = allPoints.metToGeog(surface.interpX, surface.interpY)
depthSurf = surface.interpZ

# Surface geometry (strike dip, etc.)
surface.surfaceNormal(surface.interpS, surface.interpD)
stSurf, dpSurf = normalToStrikeDip(surface.normalAprox)
surface.simpleCellWidths()
strikeWidth = surface.strikeWidth
dipWidth = surface.dipWidth

# 2d position for spline interpolation
interpSTD = surface.interpS
interpDTD = surface.interpD

# I save this as an array and load it later into a different class.
saveArrays([latSurf, lonSurf, depthSurf, stSurf, dpSurf, interpSTD, interpDTD, strikeWidth, dipWidth],
          fName = 'surfaceArraysCoarseTD', path = variable_folder)
########

## 3.2 Results

The 3D fault surface that best fits the clustered hypocenters can be seen in Figures 4 and 5. We first describe the surface. It extends 34.0 km in $x_1’$ and 15.4 km in $x_2’$. Visually speaking, the surface is smooth and does not deviate greatly from a plane. The basic plane from which the surface is built, where $x_3’=0$, has a strike and dip of 143° and 42.7°, respectively. At the south-east of the fault surface, the dip suddenly decreases going from bottom to top at a depth of about 3 km. The fault surface wavers here as it interacts with the more complex distribution of hypocenters. At the north-west of the fault surface, it is very flat where hypocenter availability was low. 

We note specifically two locations on the surface where issues in the interpolation might have arose; (i) near the south-east edge of the fault where there are conjugate faults intersecting the L’Aquila fault (Figure 5b), and (ii) at the north-west edge of the fault where there were few aftershocks for interpolation (Figure 4).

Regarding problem (i), because the clustering did not distinguish between some conjugate planes that were closely connected to the mainshock plane, the surface interpolation is slightly shifted by these conjugate fault hypocenters (Figure 5). Because the hypocenters were widely distributed near the fault in the direction normal to the fault (the aftershocks deviate from the 3D surface by an average magnitude of ~600 m), small variation in the fault surface does not conflict with the hypocenter distribution. It is likely thus not a problem that the surface is shifted slightly in this location. 

Regarding problem (ii), although the splines were inverted with regularly distributed knots, the hypocenters which were used to find the knot parameters were distributed unevenly. Particularly at the upper north-west of the fault, interpolation was done where few hypocenters were available. The spline parameters in this location are thus underconstrained. If the morphology of the fault were more complex, this problem would have to be addressed. Because the splines and fault are generally simple and nearly planar, the few available hypocenters in this region appear to be enough for our spline inversion. Further, the result of the slip distribution inversion on this fault surface (discussed later and shown in Figure 8) shows mainshock slip in this under constrained region, suggesting that although few aftershocks are found here, this part of the fault is important and cannot be removed. Further, because we ultimately compare the slip distribution on this fault surface to that of other slip models with rectangular boundaries, the comparison would be inconsistent if we removed portions of our 3D surface (Figure 8). 

To evaluate the accuracy of our 3D fault model, we compared its orientation to the orientation of earthquake focal mechanisms. This comparison is quantified as the angle between the normal vectors at the fault surface model and the normal vectors of the aftershock focal mechanisms (Figure 6). For the focal mechanisms, both the fault planes and auxiliary planes are used. For both our 3D fault and for the planar jointly inverted model (Cirella et al., 2012), there is alignment at two primary peaks of 20° and 90° which correspond to the fault and auxiliary planes of the focal mechanisms, respectively (Figure 6b). Agreement improves when the energy associated to each event $\left( \sim 10^M\right)$ is used as a weight. Note that we do not include the focal mechanism of the mainshock in this analysis because it has too much weight.

#### Figure 4
Figure 4. (a) 2D representation of the aftershocks that were part of the Paganica fault cluster, and (b) the surface that interpolates them. Position is shown on the best-fitting plane to the hypocenters. Color represents distance in meters from the best fitting plane.

In [ ]:
###! The aftershock image has been transposed! 020102020
def plotSurface(surface):
    surface.gridNodesStrike = 100
    surface.gridNodesDip = 100
    surface.splineInterp(cutPoints=True)
    cbarmin = -1e3
    cbarmax = 1e3

    matplotlib.rcParams.update({'font.size': 12})
    plt.figure(figsize=(15,6))

    xplot = (surface.pS-np.amin(surface.pS))/1000
    yplot = (surface.pD-np.amin(surface.pD))/1000
    #pS and pD are point (hypocenter) position in coordinate system of (strike, dip, normal)
    plt.scatter(xplot, yplot, 
                c = surface.pN, vmin = cbarmin, vmax = cbarmax, 
                s = 10, linewidths = .1, edgecolor = 'k') 
    plt.title('Aftershock Position')
    plt.xlabel('Strike distance (km)')
    plt.ylabel('Dip distance (km)')
    cbar = plt.colorbar()
    cbar.set_label('Distance from plane (m)')
    plt.axes().set_aspect('equal')

    shiftylabel = -2
    plt.text(np.amin(xplot), np.amin(yplot)+shiftylabel, 'NW Base', 
             verticalalignment = 'top',
             horizontalalignment = 'right')
    plt.text(np.amax(xplot), np.amin(yplot)+shiftylabel, 'SE Base', 
             verticalalignment = 'top',
             horizontalalignment = 'left')

    ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()

    plt.show()

    plt.figure(figsize=(15,6))
    ax = plt.gca()
    ax.set(xlim = xlim, ylim = ylim)
    xmin = 0
    ymin = 0
    xmax = (np.amax(surface.interpD) - np.amin(surface.interpD))/1000
    ymax = (np.amax(surface.interpS) - np.amin(surface.interpS))/1000


    xplot = (surface.interpS-np.amin(surface.interpS))/1000
    yplot = (surface.interpD-np.amin(surface.interpD))/1000
    zplot = surface.interpN
    tri = plt.tripcolor(xplot, yplot, zplot)
    tri.set_clim(cbarmin, cbarmax)
    cbar = plt.colorbar()
    cbar.set_label('Distance from plane (m)')
    plt.title('Surface Position')
    plt.xlabel('Strike distance (km)')
    plt.ylabel('Dip distance (km)')

    plt.text(ymin, xmin+shiftylabel, 'NW Base', 
             verticalalignment = 'top',
             horizontalalignment = 'center')
    plt.text(ymax, xmin+shiftylabel, 'SE Base', 
             verticalalignment = 'top',
             horizontalalignment = 'center')

    ax = plt.gca()
    # ax.set(xlim = [xmin, xmax], ylim = [ymin, ymax])


    plt.show()
plotSurface(surface)

#### Figure 5
Figure 5. Also available interactively online at https://plot.ly/~BrennanBrunsvik/12/. This shows the interpolated fault surface compared to locations of hypocenters. The images face approximately north-west. Green represents the hypocenters that were part of the cluster that contained the mainshock. Red shows all other hypocenters. (b) Shows two challenges for the fault surface interpolation. First, there are about three conjugate fault like trends in the aftershocks which were clustered as the L’Aquila fault. Second, the aftershocks and surface interpolation have low dip near the vertical top of the south-east section of the Paganica fault. This could be a separate and intersecting fault.

In [ ]:
if True or runAllFigures:
    surface.figure3DSurface(otherx=db2.x[~booLaqdb], othery=db2.y[~booLaqdb], otherz=db2.z[~booLaqdb],
                          pointSize=.7 * 2, pLineWidthR = 1/10) 
#                             ,save = True, saveName = 'Figure 5')
### This 3D figure looks the same as when things worked, 020102020

In [ ]:
# Make copy of old surface object so it doesn't get broken
# use it for comparing focal mechanisms. One for our surface + one for Cirella et al. (2012)

# Compare focal to a the planar fault plane of Cirella
planeVsSurface = copy.deepcopy(surface)
# input strike, dip, rake, get normals, use these for comparison
altSurfaceNormal, __=faultUnitVectors(np.array([2.32128791]),
                 np.array([0.9424778]), 
                 np.array([0])
                )
planeVsSurface.compareFocal(altSurfaceNormal=altSurfaceNormal)



# Compare focal to our 3D surface
unmodified = copy.deepcopy(surface)
unmodified.compareFocal()

# the dot products has been completed.

# create weights for Figure 5
mag = db2.mL[db2.labels==0][unmodified.focalAvailable]
weights = 10 ** mag 
bN = weights!=np.amax(weights)  # Boolean which excludes mainshock because it otherwise has too much weight

#### Figure 6
Figure 6. The angle between the normal of the interpolated fault surface and the normal of the available aftershock focal mechanisms. (a) Orientation compared to location of hypocenters in the cluster that corresponds to the L’Aquila fault. Location is distance along strike and along dip on the fault surface. Only the best matching angle for each aftershock is used for (a). (b) Histograms of the angles from the normal of the fault surface to the normals of the aftershock planes. The histograms are normalized so that their integrals are 1. We show both our 3D fault model and the planar model of Cirella et al. (2012). Both potential planes of each focal mechanism were included. This results in alignment peaks near 90° that correspond with the auxiliary planes of focal mechanisms. For the fault planes of the focal mechanisms, alignment peaks are closer to 20°. We show histograms both as unweighted and as weighted by 10M. Orientation match is increased when this is done. 

In [ ]:
# Figure 6a
### ! the dip distance has flipped since previous submission: 02012020, as well as the distance being multiplied by -1
matplotlib.rcParams.update({'font.size': 12})
boo = unmodified.focalAvailable
plt.figure(figsize=(15,7))
plt.axes().set_aspect('equal')

x = (unmodified.pS[boo]-np.amin(unmodified.pS) )/1000
y = (unmodified.pD[boo]-np.amin(unmodified.pD) )/1000

plt.scatter(x, y, c=unmodified.thetaLeast*180/np.pi,
        vmin = 0, vmax = 90, linewidths=.3, edgecolors='k',
           s = 15)#3**mag)

shiftylabel = -2
plt.text(np.amin(x), np.amin(y)+shiftylabel, 'NW Base', 
         verticalalignment = 'top',
         horizontalalignment = 'right')
plt.text(np.amax(x), np.amin(y)+shiftylabel, 'SE Base', 
         verticalalignment = 'top',
         horizontalalignment = 'left')

cbar = plt.colorbar()
cbar.set_label('Angle $\degree$ from 3D surface')
plt.xlabel('Strike distance (km)')
plt.ylabel('Dip distance (km)')
plt.title('Orientation Mismatch')

plt.show()

In [ ]:
matplotlib.rcParams.update({'font.size': 11})
fig, axs = plt.subplots(2, 2, figsize=(12,7))

axs[0, 0].hist(
    np.array([unmodified.theta1[bN], unmodified.theta2[bN]]).ravel()*180/np.pi, 
         bins = 50, range=(0, 180), density = True # Unmodified has the wrong theat 02012020
        )
axs[0, 1].hist(np.array([planeVsSurface.theta1[bN], planeVsSurface.theta2[bN]]).ravel()*180/np.pi, 
         bins = 50, range=(0, 180), density = True
        )
axs[1,0].hist(np.array([unmodified.theta1[bN], unmodified.theta2[bN]]).ravel()*180/np.pi, 
         bins = 50, range=(0, 180), density=True,
         weights=np.array([weights[bN], weights[bN]]).ravel()
         )
axs[1,1].hist(np.array([planeVsSurface.theta1[bN], planeVsSurface.theta2[bN]]).ravel()*180/np.pi, 
         bins = 50, range=(0, 180), density = True,
         weights=np.array([weights[bN], weights[bN]]).ravel()
        )

rangeTopFig = (0, .015)
rangeBotFit = (0, .05)
for i in range(2):
    axs[0, i].set_ylim(rangeTopFig)
for i in range(2):
    axs[1, i].set_ylim(rangeBotFit)

for i in range(2): 
    for j in range(2):
        axs[i, j].tick_params(labelleft=False) 

axs[0,0].set_title('3D Model', y = 1)
axs[0,1].set_title('Planar Model', y = 1)

for i in range(2):
    axs[1, i].set_xlabel('Angle')
    
axs[0, 0].set_ylabel('Mechanisms, Unweighted')
axs[1, 0].set_ylabel('Mechanisms, Weighted $M^{10}$')

for i in range(axs.shape[0]):
    for j in range(axs.shape[1]):
        axs[i, j].set_xticklabels(['{:.1f}$\degree$'.format(x) for x in plt.gca().get_xticks()])

plt.tight_layout(pad = 1.3)
fig.suptitle("Angle from Focal Mechanism to Mainshock Model", fontsize="x-large", y = 1.05)
plt.show()

In [ ]:
#02012020 this test indicates that the normal vectors are about right for the 3d surface. 
# I think the problem is just with the positions of points. 
altSurfaceNormal

n = unmodified.surfaceNormal()
np.average(n, axis = 0)

In [ ]:
# cl = unmodified
# t1keep = cl.theta1 < cl.theta2
# t = cl.theta2
# t[t1keep] = cl.theta1[t1keep]
# plt.hist(t * 180 / np.pi, bins = 90)
# plt.show()

# 4 Slip Inversion

## 4.1 Methods

We invert two co-seismic slip distribution models for the MW 6.1 L’Aquila mainshock: one slip distribution on our reconstructed 3D fault model, and another slip distribution on a 2D planar fault model. This allows for a direct evaluation of how the 3D fault morphology affects Coulomb stress. Slip distributions are inverted only with co-seismic GPS deformation measurements. GPS data includes 77 3-component GPS stations, both survey and continuous (Serpelloni et al., 2012). To calculate ground displacement, we use a Poissonian medium with the Lamé parameters equal to 26 GPa as in Serpelloni et al. (2012; see Di Luzio et al., 2009 for crustal model). 

Similar to Serpelloni et al. (2012), we invert for a planar fault with a homogenous slip distribution in order to obtain the fault planes location and orientation. This location and orientation is used for our planar fault with heterogeneous slip distribution. For the homogenous slip distribution inversion, we attempt to find the global minimum of the weighted residual sum of squares, $WRSS$, between the observed displacements, $d_o$, and modelled displacements, $d_m$, where
$$WRSS = \left( d_{o}−d_{m} \right)^{T}\cdot cov^{−1} \cdot \left( d_{o}−d_{m} \right)$$
and $cov$ is the covariance matrix of the GPS measurements. $cov$ is made assuming both that there is no correlation between measurements at different GPS stations, and there is no correlation between measurements in different directions at the same station. Thus, it is a diagonal matrix with only the variance of each GPS data. dm for this model and all planar models was calculated using homogeneous half-space cells (Okada, 1985; 1982) with the original Fortran code implemented as a Python wrapper (Thompson, 2017). We use the Basin-Hopping minimization algorithm from Scientific Python to find a global minimum WRSS for the non-linear problem (Jones et al., 2014). The parameters were simultaneously inverted: latitude, longitude, depth (positions are for the center of the fault), strike, dip, rake, slip, length in strike direction, and width in dip direction (Table 1). 

When inverting for the slip distribution on our 2D planar fault, it is helpful to increase the size of the fault to allow for slip to gently taper off toward the edges of the fault. The length of the fault in the strike direction was extended to 35 km, similar to the length of the inverted 3D fault surface. The width of the fault was left unmodified because the planes maximum depth (about 12 km) already extends beyond the maximum observed depth of hypocenters on the fault (about 11 km). 

To invert for slip distributions on both the planar and 3D fault models, the methods are very similar to each other except that the individual fault cells are dealt with differently (discussed later). We represent the slip distribution on faults using bi-cubic splines (Jones et al., 2014). A grid of nodes is distributed over the fault with variable slips, and the splines are used to interpolate for the slip between these nodes. Slip values at the base and sides of the fault are held at 0. Because the ground surface was co-seismically ruptured (Boncio et al., 2010; Roberts et al., 2010; Cinti et al., 2011), slip is allowed to vary at the top of the fault. There are 9 by 6 spline nodes in the strike and dip direction respectively. For the planar fault, 40 by 24 uniformly sized Okada (1985, 1992) type fault patches are distributed over the fault, and the slip on these patches is calculated based on the slip value of the splines in the centers of the fault patches. For the 3D fault, we test a similar 40 by 24 grid of Okada cells, but these rectangular cells cannot be meshed in 3D without creating gaps in the fault. Thus, we also use triangular dislocations (Nikkhoo and Walter, 2015) based on a Delaunay of 40 by 24 points (all results associated with the 3D fault model come from the triangular solution and not the Okada solution unless otherwise specified). The slip distributions inverted using Okada cells compared to triangular dislocations are only negligibly different. Stresses that are later found with these different methods (Figure 11) are also negligibly different with end results varying by ~1%. 

To prevent an unrealistically sharp slip distribution which overfits the data, some smoothing function is needed. For the fault models with variable slip distribution, instead of simply minimizing for $WRSS$, we minimize $WRSS+R$ where $R$ is a positive scalar that quantifies the roughness of the slip distribution. $R$ is defined here as:
$$R = \sum_{i=1}^{2} \int \left( \nabla ^2 u_{i} \right)^{2} dA$$
where $u_i$ is the slip in the strike slip or dip slip direction and  is a smoothing factor that is manually chosen to determine how important the roughness of the slip distribution is relative to minimizing $WRSS$. There are many ways to quantify roughness. We chose a function based on Harris and Segal (1987). We chose  where there is a maximum trade-off between roughness and $WRSS$ (Figure S1). The choice of  is subjective here, but statistical methods can apply smoothing in a more objective way (Yabuki and Matsu'ura, 1992; Cambiotti et al., 2017). We used the BFGS minimization algorithm to find minimums of $WRSS+R$ (Jones et al., 2014).

### Invert for mainshock model with homogenous slip

In [ ]:
# Starting point for inverting fault with homogenous slip distribution
# Starting parameters should be quite close to final ones, as the problem is 
#     non linear and difficult. This isn't an important point of our paper. 

Strike                   = np.array([ 2.259223])
Dip                      = np.array([ 0.890402])
Rake                     = np.array([-1.822806])
Lon                      = np.array([ 13.42772])
Lat                      = np.array([ 42.31982])
Depth                    = np.array([-7072.831])
Slip                     = np.array([ 0.601985])
Strikeslip               = np.array([-0.150105])
Dipslip                  = np.array([-0.582970])
Tensile                  = np.array([ 0.      ])
Strikelength             = np.array([ 13842   ])
Diplength                = np.array([ 16098   ])
Strikerectangles         = np.array([ 1       ])
Verticalrectangles       = np.array([ 1       ])

# make object for homogenous slip distribution inversion
hom = inversionHom(iterations=1000, arraySaveName=variable_folder+'homogenousTests', rescale=True, method='CG')
hom.getStartVals(Lon, Lat, Depth, Strike, Dip, Rake, Slip, 
           Strikelength, Diplength)

In [ ]:
if True:
    hom.loadResults()

hom.iterations = 10
if False:
    hom.runOptimizeBH(ratioOfEpsilon=.2)

if False:
    hom.saveResults()

In [ ]:
hom.testCS() # testCS here really just is used to find displacements at GPS stations
hom.plotErrors()

#### Table 1. 
Table 1. The 9 parameters that were inverted to represent a rectangular fault model with homogenous slip for the L’Aquila mainshock. Parameters were inverted by matching modelled and observed co-seismic displacements at GPS stations. This position and orientation was then used to invert the slip distribution for our 2D fault model.

In [ ]:
# Table 1
if True: 
    homLabels = ['Strike', 'Dip', 'Rake', 'Lon', 
                 'Lat', 'Depth (km)', 'Slip (cm)',
                 'Strike Length (km)', 'Dip Length (km)']

    homValsTable = [
        np.around(hom.s*180/np.pi), 
        np.around(hom.d*180/np.pi), 
        np.around(hom.r*180/np.pi), 
        np.around(hom.lon, 3),     
        np.around(hom.lat, 3), 
        np.around(-hom.depth/1000, 2), 
        np.around(hom.slip*100, 1),
        np.around(hom.strikeLength/1000, 1), 
        np.around(hom.dipLength/1000, 1)
                  ]

    for i, thing in enumerate(homValsTable):
        homValsTable[i] = float(thing)

    values = [homLabels, homValsTable]
    trace = go.Table(cells = dict(values = values))
    data = [trace]
    layout = dict(width=500)
    fig = dict(data=data, layout=layout)
    plot(fig)

In [ ]:
"""This cell is just a test to make sure the code works.
I test displacement and GPS station errors using a known fault 
model from Serpelloni et al. (2012).

Within some range of error with varying geographic projects etc., 
our calculations seem to work."""

ser = inversionHom("Serpelloni's homogenously inverted parameters.")

ser.s = np.array([129.4 * np.pi / 180])
ser.d = np.array([50.5 * np.pi / 180])
ser.r = np.array([-105.0 * np.pi / 180.])

ser.lon = np.array([13.387])
ser.lat = np.array([42.285])
ser.slip = np.array([.602]) 

ser.strikeLength = np.array([13900])
ser.dipLength = np.array([16000]) 
ser.depth = - np.array([13200])

ser.strikeRects = np.array([1])
ser.verticalRects = np.array([1])

ser.centerOrigin='bottomCenter'
ser.slipComponents()
ser.makeLists() # make list of these parameters for later code. 

ser.testCS()
ser.plotErrors()

### Prepare Cirella et al. (2012) model

In [ ]:
cir = bilinearFaultImport()
cir.loadModel(splineShape=(6, 9))
cir.shiftKnots()
cir.getMainshockPosition()
cir.spGrid()

cir.mechanicalConstants()
if True:
    cir.GPSConstants()
    print('use GPSConstantsAlternate to use GPS data more similar to what was used in Cirella et al. (2012)')
if False:
    cir.GPSConstantsAlternate()
cir.dummyPointSDR()

cir.strikeRects = 40 
cir.verticalRects = 24 

cir.makeLists()
cir.geogToMet()
cir.coord2d()
    
cir.spGrid()
cir.testCS()

In [ ]:
cir.plotErrors()

In [ ]:
cir.spImageVector()

### Heterogeneous slip planar fault

In [ ]:
# Multiple iterations with increasing number of nodes.
heter = inversions()
heter.importHomogenous(hom) # get geometry of previously inverted fault
heter.getMainshockPosition() # project mainshock into a strike-dip coordinate system for plots

# stretch strike length to accomadate for tapering fault
heter.strikeLength = np.array([35e3])

heter.mechanicalConstants()
heter.GPSConstants()
heter.dummyPointSDR()

heter.strikeRects = 40
heter.verticalRects = 24
heter.setMinMax()

heter.makeLists()
heter.geogToMet()
heter.ratio=2

heter.coord2d()
heter.sNodes = 9
heter.dNodes = 6
heter.spGrid()

heter.spSlipsInit()
heter.splineToCells()
heter.prepLaplacian()

In [ ]:
# Load saved results, or run again. 
if False:
    heter.constructLinearArrays()
    
heterLinearArrays = variable_folder+slip_dist_folder+'heterLinearArrays'

if False:
    heter.saveLinearArrays(heterLinearArrays)

if True:
    heter.loadLinearArrays(heterLinearArrays)

In [ ]:
if True:
    iterationsEach = 100
    
    numOfb = 30
    ball = 1e7 * 10**np.linspace(4, 0, numOfb) # for loading use 1e7 * 10**np.linspace(4, 0, numOfb) with numOfb = 30

    heterBErr = [[],[],[]] # for comparing b to error


    for i in np.arange(ball.size):
        heter.b = ball[i]

        heter.saveName = slip_dist_folder+'heter_slips_b_'+'%e' % heter.b # another folder is appended in class

        if False:
            heter.tol = .0024
            heter.iterations=iterationsEach
            heter.runErrHetBFGS()

        if False:
            heter.savesp() #8x4, 16x8

        if True:
            heter.loadsp()
            
        heter.splineToCells()
        heter.getDisp()
        heter.laplacianGrid()
        heter.getWRSS()
        heterBErr[0].append(heter.b); heterBErr[1].append(heter.WRSS); heterBErr[2].append(heter.smoothe)

In [ ]:
# part of figure s1

plt.rcParams.update({'font.size':12})
plt.figure(figsize=(7, 7))
a = np.array(heterBErr)
roughness =a[2]/a[0]*1e16 
error = a[1]


plt.scatter(roughness, error, c = 'k')

indChoice = 15
plt.annotate(s = r'$\beta$ = '+'%e' % a[0, indChoice], xy = (roughness[indChoice], error[indChoice]))

plt.scatter(roughness[indChoice], error[indChoice], c = 'red')

plt.xlabel('Roughness')
plt.ylabel('WRSS')

plt.title('Tradeoff Curve Planar Fault')
plt.show()

In [ ]:
# Load the model with chosen b value
heter.b = ball[indChoice] 
heter.saveName = slip_dist_folder+'heter_slips_b_'+'%e' % heter.b # another folder is appended in class
heter.loadsp()
heter.splineToCells()
heter.getDisp()

In [ ]:
heter.plotErrors()

In [ ]:
heter.spImageVector(vmax=1.2, figsize = (15, 15))

In [ ]:
heterF = copy.deepcopy(heter) # make a copy of that object so we don't mess it up

### Invert slip on 3D fault surface using OKADA
This is just to compare to the triangular dislocation results. The Okada based results are not reused for the 3D surface results. 

In [ ]:
surf = inversionCurved(fName = 'surfaceArraysCoarse')

surf.saveName='surface_spline_slips'
surf.savePath=variable_folder

surf.temporarySplineBoundary()
surf.spGrid(9, 6)
surf.spSlipsInit(ss=-.122, ds=-.495) # Start with values of homogenous inversion

surf.getMainshockPosition()
surf.makeListInitial()
surf.importShapeSplines(surface)
surf.prepLaplacian()

In [ ]:
# Load saved results, or run again. 
if False:
    surf.constructLinearArrays()
    
surfLinearArrays = variable_folder+slip_dist_folder+'surfLinearArrays'

if False:
    surf.saveLinearArrays(surfLinearArrays)

if True:
    surf.loadLinearArrays(surfLinearArrays)

In [ ]:
if True:
    iterationsEach = 100
    
    numOfb = 30
    ball = 1e7 * 10**np.linspace(4, 0, numOfb)
    

    surfBErr = [[],[],[]] # for comparing b to error


    for i in np.arange(ball.size):
        surf.b = ball[i]

        surf.saveName = slip_dist_folder+'surf_slips_b_'+'%e' % surf.b # another folder is appended in class

        if False:
            surf.tol = .0024
            surf.iterations=iterationsEach
            surf.runErrHetBFGS()

        if False:
            surf.savesp() #8x4, 16x8

        if True:
            surf.loadsp()
            
        surf.splineToCells()
        surf.getDisp()
        surf.laplacianGrid()
        surf.getWRSS()
        surfBErr[0].append(surf.b); surfBErr[1].append(surf.WRSS); surfBErr[2].append(surf.smoothe)

In [ ]:
plt.rcParams.update({'font.size':12})
plt.figure(figsize=(7, 7))
b = np.array(surfBErr)
roughness =b[2]/b[0]*1e16 
error = b[1]


plt.scatter(roughness, error, c = 'k')

indChoice = 15
plt.annotate(s = r'$\beta$ = '+'%e' % b[0, indChoice], xy = (roughness[indChoice], error[indChoice]))
plt.scatter(roughness[indChoice], error[indChoice], c = 'red')


plt.xlabel('Roughness')
plt.ylabel('WRSS')

plt.title('Tradeoff Curve 3D Fault')
plt.show()

In [ ]:
# Load the model with chosen b value
surf.b = ball[indChoice]
surf.saveName = slip_dist_folder+'surf_slips_b_'+'%e' % surf.b # another folder is appended in class
surf.loadsp()
surf.splineToCells()
surf.getDisp()

In [ ]:
surf.plotErrors()

In [ ]:
surf.spImageVector(vmax=1.2, figsize = (15, 15))

In [ ]:
surfF = copy.deepcopy(surf) # Make copy of object so we don't mess it up

### Invert slip on 3D fault surface using Triangular Dislocations.

In [ ]:
surfTD = inversionCurvedTD(fName = 'surfaceArraysCoarseTD', cellType = 'TD')

In [ ]:
surfTD.saveName='surface_spline_slipsTD'
surfTD.savePath=variable_folder

surfTD.temporarySplineBoundary()
surfTD.spGrid(9, 6)
surfTD.spSlipsInit(ss=-.122, ds=-.495) # Start with values of homogenous inversion

In [ ]:
surfTD.getMainshockPosition()
surfTD.makeListInitial()
surfTD.importShapeSplines(surface)
surfTD.prepLaplacian()

In [ ]:
# Load saved results, or run again. 
surfTDLinearArrays = variable_folder+slip_dist_folder+'surfTDLinearArrays'
if False: 
    surfTD.saveSimplicesPermanent()
    surfTD.constructLinearArraysTD()
    
if False: 
    surfTD.saveLinearArrays(surfTDLinearArrays)

if True:
    surfTD.loadLinearArrays(surfTDLinearArrays)
    surfTD.loadSimplices()

In [ ]:
if True:
    iterationsEach = 100
    
    numOfb = 30
    ball = 1e7 * 10**np.linspace(4, 0, numOfb)
    

    surfTDBErr = [[],[],[]] # for comparing b to error


    for i in np.arange(ball.size):
        surfTD.b = ball[i]

        surfTD.saveName = slip_dist_folder+'surfTD_slips_b_'+'%e' % surfTD.b # another folder is appended in class

        if False: 
            surfTD.tol = .0024
            surfTD.iterations=iterationsEach
            surfTD.runErrHetBFGS()

        if False: 
            surfTD.savesp()

        if True: 
            surfTD.loadsp()
            
        surfTD.splineToCells()
        surfTD.getDisp()
        surfTD.laplacianGrid()
        surfTD.getWRSS()
        surfTDBErr[0].append(surfTD.b); surfTDBErr[1].append(surfTD.WRSS); surfTDBErr[2].append(surfTD.smoothe)

In [ ]:
# Part of figure s1

plt.rcParams.update({'font.size':12})
plt.figure(figsize=(7, 7))
b = np.array(surfTDBErr)
roughness =b[2]/b[0]*1e16 
error = b[1]


plt.scatter(roughness, error, c = 'k')

indChoice = 15
plt.annotate(s = r'$\beta$ = '+'%e' % b[0, indChoice], xy = (roughness[indChoice], error[indChoice]))
plt.scatter(roughness[indChoice], error[indChoice], c = 'red')


plt.xlabel('Roughness')
plt.ylabel('WRSS')

plt.title('Tradeoff Curve 3D Fault')
plt.show()

In [ ]:
# Load the model with chosen b value
surfTD.b = ball[indChoice]
surfTD.saveName = slip_dist_folder+'surfTD_slips_b_'+'%e' % surfTD.b # another folder is appended in class
surfTD.loadsp()
surfTD.splineToCells()
surfTD.getDisp()

In [ ]:
surfTD.plotErrors()

surfTD.spImageVector(vmax=1.2, figsize = (15, 15))

In [ ]:
surfTDF = copy.deepcopy(surfTD) # Make copy of object so we don't mess it up

## 4.2 Results

(Note that Figure 6 and 7 are repeated here with parts a-c combined for easy comparison while these sub-parts were seperated in the methods section)

The final results for the fault model inversion with a homogenous slip distribution can be seen in Table 1. For a comparison to other planar models, see Serpelloni et al. (2012) and references therein. Our parameters are similar to those obtained from other studies. 

Misfit between modelled and GPS measured co-seismic displacements can be seen in Figure 7 and Table 2. Inverted slip distributions can be seen in Figure 8. For comparison, we include the jointly inverted planar model of Cirella et al. (2012) and approximate their model using a grid of homogenous half-spaces (Okada, 1985, 1992). Because the joint model was originally inverted using a 3D mechanical model, our half-space approximation may become misrepresentative for stations close to the fault. Further, because of errors in GPS measurements, the inclusion of strong-motion and InSAR data decreases displacement fit at GPS stations even though the model becomes more realistic. The jointly inverted planar model thus has a slightly higher misfit than the purely GPS inverted models. For the purely GPS based inversions, we find that our 3D mainshock model fits worse for horizontal data and better for vertical data compared to our planar model. 


#### Figure 7
Figure 7. Modelled (blue) and observed (red) displacements at GPS stations produced from the jointly inverted model by Cirella et al. (2012), our 2D planar model with GPS based slip distribution inversion, and our 3D model with GPS based slip distribution inversion.

In [ ]:
def plotDxyz(ax, lat, lon, dxO, dxM, dyO, dyM):
    """plot vectors dx and dy (observed and modeled). 
    
    lat and lon are position of vector tails.
    dx and dy are x and y displacements.
    O or M signifies observed and modelled values."""
    ones = np.ones(lat.shape)
    lat = lat * ones
    lon = lon * ones
    dxO = dxO * ones
    dxM = dxM * ones
    dyO = dyO * ones
    dyM = dyM * ones
    
    scalePlot = .65
    
    edgecolor='black'
    linewidth=.5
    ax.set_xlim(13.1, 13.8)
    ax.set_ylim(42.1, 42.6)
    
    #scale bar
    xbar = np.array([13.2])
    ybar = np.array([42.15])
    barup = np.array([.05])
    barside = np.array([0])
    co = np.ones(lon.size, dtype = str)
    cm = np.ones(lon.size, dtype = str)
    cb = np.ones(1, dtype = str)
    co[:] = 'r'
    cm[:] = 'b'
    cb[:] = 'k'
    
    xall = np.append(np.append(lon, lon), xbar)
    yall = np.append(np.append(lat, lat), ybar)
    dxall= np.append(np.append(dxO, dxM), barside)
    dyall= np.append(np.append(dyO, dyM), barup)

    colors = np.append(np.append(co, cm), cb)
    
    ax.quiver(xall, yall, dxall, dyall, color = colors,
             scale_units = 'x', scale = scalePlot,
             angles = 'uv', edgecolor=edgecolor, linewidth=linewidth)
    
    ax.annotate('  5 cm', (xbar, ybar))
    
    # A scatter plot to show vector tail locations
    ax.scatter(lon, lat, c='k', s=5)

In [ ]:
def plotErrors(self, label=None):
#     fig = plt.figure(figsize=(15, 15))
#     axs = fig.add_subplot(122, aspect='equal')
    fig, axs = plt.subplots(1, 2)#, sharex=True)
    fig.set
    fig.subplots_adjust(left=0, right=2, bottom=0, top = 1)
#     fig.subplots
#     axs.reshape()
    if label is not None:
        axs[0].set_ylabel(label)
    axs[0].set_title('Horizontal (mm)')
    axs[1].set_title('Vertical (mm)')
    plotDxyz(axs[0], self.latP, self.lonP, self.dxMeas, self.dx, self.dyMeas,self.dy)
    plotDxyz(axs[1], self.latP,self.lonP, 0, 0, self.dzMeas, self.dz)

    for i in range(2):
        axs[i].set_yticklabels(['{:.1f}$\degree$'.format(x) for x in plt.gca().get_yticks()])
        axs[i].set_xticklabels(['{:.1f}$\degree$'.format(x) for x in plt.gca().get_xticks()])

    plt.show()
    print('rms residual for dx, dy, dz = ',rmsResiduals(self.dxMeas, self.dyMeas, self.dzMeas, 
                 self.dx, self.dy, self.dz) )

In [ ]:
# Figure 7

if False or runAllFigures:
    matplotlib.rcParams.update({'font.size': 12})
    mods = [cir, heterF, surfTDF]
    titles = ['Planar Joint', 'Planar GPS', '3D GPS']

    for i, model in enumerate(mods):
    #     fig, ax = fig, axs = plt.subplots(4, 2, sharex=True)
#         if titles[i] != '3D GPS':
#             model.testCS()
#         else:
#             model.testCSTD()
        model.plotErrors = plotErrors
        model.plotErrors(model, label = titles[i])

#### Table 2
Table 2. Same error as in Figure 7. Values are RMS residual mismatch between observed and modelled co-seismic displacements (mm).

In [ ]:
if False or runAllFigures:

    # Table 2
    models = [cir, heterF, surfTDF]
    labels = ['Planar Joint', 'Planar GPS', '3D GPS']
    errors = np.zeros((len(models), 3))
    for i, mod in enumerate(models):
        if labels[i] != '3D GPS':
            model.testCS()
        else:
            model.testCSTD()
        xerror, yerror, zerror = rmsResiduals(
            mod.dx, mod.dy, mod.dz, mod.dxMeas, mod.dyMeas, mod.dzMeas)
        errors[i,:] = np.array([xerror, yerror, zerror])


        errorsplot = np.around(errors.T*1000, decimals=2).tolist()
        errorsplot.insert(0, labels)

        # layout = go.Layout(title='Title')
    trace = go.Table(
        header=dict(values=['Model', 'East Error (mm)', 'North Error (mm)', 'Vertical Error (mm)']),
        cells=dict(values= errorsplot ) )
    layout = dict(width=800, height=300)
    data = [trace] 
    fig = dict(data=data, layout=layout)
    plot(fig)

#### Figure 8
Figure 8. These are the slip distributions we consider for the 2009 L’Aquila mainshock. Because each rupture model is in a different location, the mainshocks hypocenter should be used as a reference when comparing slip distributions. The location of this hypocenter is represented with a star. Vectors show co-seismic displacement on the hanging wall relative to the footwall. (a) Slip distribution jointly inverted by Cirella et al. (2012), (b) slip distribution on our inverted 2D fault plane (c) slip distribution inverted with our 3D fault.

In [ ]:
# Figure 8

matplotlib.rcParams.update({'font.size': 16})
mods = [cir, heterF, surfTDF]
titles = ['Planar Joint', 'Planar GPS', '3D GPS']
for i, mod in enumerate(mods):
    def spImageVector(self, xVectors=12, yVectors = 6, xPoints=200, 
                      yPoints=100, showMainshock=True, vmin=0, vmax = 1.2, 
                      shiftLabels=True, figsize = (15, 15), title = None ):
        """ replacing old spline to cells, using interp2d"""
        fig = plt.figure(figsize=figsize)
        self.splineToCells()

        xLin = np.linspace( self.minS, self.maxS, 
                                 num=xPoints)
        xLoc = np.outer( xLin, np.ones(yPoints) )

        yLin = np.linspace( self.minD, self.maxD, 
                                 num=yPoints)
        yLoc = np.outer( np.ones(xPoints), yLin )



        xLinV = np.linspace( self.minS, self.maxS, 
                                 num=xVectors)
        xLocV = np.outer( xLinV, np.ones(yVectors) )

        yLinV = np.linspace( self.minD, self.maxD, 
                                 num=yVectors)
        yLocV = np.outer( np.ones(xVectors), yLinV )


        ssIm = self.rbfss(xLin, yLin)
        dsIm = self.rbfds(xLin, yLin)

        ssImV = self.rbfss(xLinV, yLinV)
        dsImV = self.rbfds(xLinV, yLinV)

        extent = np.array([np.amin(xLin), np.amax(xLin), np.amin(yLin), np.amax(yLin)])
        if shiftLabels:
            extentShift = np.array([-self.mainS, -self.mainS, -self.mainD, -self.mainD]).ravel()
            extent+=extentShift
            xLocV-=self.mainS
            yLocV-=self.mainD
            
            extent*=1/1000
            xLocV*=1/1000
            yLocV*=1/1000

        im = plt.imshow(np.sqrt(ssIm.T**2+dsIm.T**2), cmap='YlOrBr', 
                   origin='lower', extent=extent, vmin = vmin, vmax = vmax,
                   interpolation='bicubic')
        if showMainshock:
            if shiftLabels:
                mainS=0
                mainD=0
            elif not shiftLabels:
                mainS = self.mainS
                mainD = self.mainD
            try:
                plt.scatter(mainS, mainD, marker="*", s=500, c='yellow', linewidths=1, edgecolors='k')
            except: print('No mainshock position')
        plt.quiver(xLocV, yLocV, ssImV, dsImV, scale=10)
        
        cb = plt.colorbar(im, orientation='horizontal', fraction=0.026, pad=0.06)
        
        cb.set_label('Slip (m)')
        plt.xlabel('Strike Distance (km)')
        plt.ylabel('Dip Distance (km)')
        yshift = -.75
        plt.text(np.amin(xLocV), np.amin(yLocV)+yshift, 'NW Base', 
                 verticalalignment = 'top',
                 horizontalalignment = 'center')
        plt.text(np.amax(xLocV), np.amin(yLocV)+yshift, 'SE Base', 
                 verticalalignment = 'top',
                 horizontalalignment = 'center')
        
        if not title is None:
            plt.title(title)#, loc = 'right')
        
        plt.tight_layout()
        plt.show()
    mod.spImageVector = spImageVector
    mod.spImageVector(mod, title = titles[i])

# 5 Coulomb Stress

## 5.1 Methods

Static stress change is believed to be one of the main causes of earthquake interaction, and ∆CFF is commonly used to quantify the promotion of fault failure due to static stress change (e.g. Stein, 1999 and references therein). To understand the significance of incorporating an earthquake 3D morphology into stress calculations, and to better understand earthquake interaction and the source process for the L’Aquila seismic sequence and earthquakes in general, we compare the stresses that result from each mainshock model. We use a dataset with 3422 focal mechanisms (Aldersons et al., 2009) in order to find the ∆CFF that the mainshock generated. These focal mechanisms were calculated using the FPFIT method (USGS Open-File Report 85-739), which finds a source model which minimizes first-motion polarity discrepancies. 

Coulomb stress is defined as
$$CFF=\tau+\mu \left(\sigma_n+P\right)$$ 
where $CFF$ is the Coulomb stress, $\tau$ is the shear stress across the fault in the direction of slip, $\mu$ is the coefficient of friction across the fault, $\sigma_n$ is the normal stress on the fault (we use positive as tension), and P is the pore pressure. $CFF$ can be used for quantitative analysis showing how much the stress on a fault contributes to rupture. This compares the maximum sustainable shear stress on a fault to the actual shear stress where positive $CFF$ suggests fault failure.  Because we only look at how the earthquake contributes to failure, we find
$$\Delta CFF = \Delta \tau + \mu \left( \Delta \sigma_n + \Delta P \right)$$

To account for pore pressures response to volumetric change, the change in pore pressure is treated as proportional to the volumetric stress, 
$$ \Delta P = − B \frac{tr\left(\Delta \sigma\right)}{3} $$
, where $B$ is Skempton’s coefficient (Cocco and Rice, 2002; Makhnenko and Labuz, 2013). This formula is sufficiently accurate while not introducing many new, unknown constants. We assumed $B=0.45$ and $\mu = 0.6$. Within reasonable variation, stress results are only minimally sensitive to the values of $B$ and $\mu$. Various sets of constants have been used for the L’Aquila sequence. For instance, Terakawa et al. (2010) used $\mu = 0.6$ and $B=0.6$, Serpelloni et al. (2012) used $\mu= 0.75$ and $B = 0.47$, Walters et al. (2009) used an effective coefficient of friction $\mu ‘ = 0.6$ (which takes both $\mu$ and $B$ into account), but had similar results with $\mu ‘ = 0.4$. 

In order to find ∆CFF, the strike, dip, and rake of a fault must be known. Because it is generally not possible to distinguish between the fault plane and auxiliary plane in an aftershock’s focal mechanism, we calculate ∆CFF on both planes as in Nostro et al. (2005). We analyze whether ∆CFF results are positive on neither, only one, or both planes. This prevents the need to assume that one plane in a focal mechanism is the true fault plane; such a choice could easily be wrong. However, care must be taken to interpret our results when ∆CFF is calculated to be positive on only one plane in a focal mechanism as that plane may not be the actual fault plane. 

We test ∆CFF both on the actual focal mechanisms of aftershocks (Aldersons et al., 2009) and on planes that are optimally oriented for failure (OOPs). There are several advantages to analyzing OOPs: (i) the actual focal mechanism inversion process has errors which could be systematic, (ii) there are many well located events (47,917, about 93% of the catalog) which do not have focal mechanisms which can be analyzed only if we assume OOPs, (iii) the spatial distribution of events, and thus sampled ∆CFF, is greater if more events are used, and (iv) smaller events which tend not to have focal mechanisms are generally better located than large events due to the complexities in cross-correlating the waveforms of large events. The orientations of OOPs are found by maximizing ∆CFF given the combination of regional stress and mainshock induced stress (King, Stein, and Lin, 1994). We adopt the regional stress given by Chiaraluce et al. (2003). We also find ∆CFF on a grid of OOPs for visualization. The orientations of these OOPs can be seen in Figure S2. 

There are also advantages to using only the actual focal mechanisms. (i) Inaccuracies in regional stress knowledge affect OOP orientations. This can be orientation error, but both the magnitude of the deviatoric stress and the depth dependence of the lithostatic stress are poorly known. (ii) Failure orientation is not only controlled by stress, but also pre-existing weaknesses and rheology. For instance, thrust faults that exist from the complex evolution of stress in the Apennines (Malinverno and Ryan, 1986; Mariucci et al., 1999; Jolivet and Faccenna, 2000) may activate at orientations much different than OOPs. The combination of testing ∆CFF on OOPs and actual focal mechanisms provides the most complete picture of ∆CFF development.  

Although induced stress is not constant in time (e.g. Verdecchia et al., 2018) we assumed that induced stress does not change after it is generated. We test this assumption by modelling stresses generated not only from the mainshock, but also from several large aftershocks (Figure S3). We find that aftershock slip distributions are not constrained enough to find reliable results. Thus, incorporating their stresses decreases modelled ∆CFF before rupture. 


In [ ]:
# first, define some functions for finding OOP orientations
def plungeTrendToUnitVec(plunge, trend):
    """Radians ALWAYS!"""
    
    x = np.cos(plunge) * np.sin(trend)
    y = np.cos(plunge) * np.cos(trend)
    z = np.sin(plunge)
    
    return np.array([x, y, z]).T

def hydroStaticStress(row, g, z):
    points = z.size
    
    meanStress = row * g * z
    if (meanStress > 0).any():
        print("shouldn't all mean stress be < 0?")
        
    idents = np.zeros((points, 3, 3))  
    ident = np.identity(3)
    
    for i in np.arange(points):
        idents[i] += ident * meanStress[i]
    
    return idents

def rotateTensFull(ten, Q):
    """Q is array of orthogonal unit vectors of new coordinate system"""
    if ten.shape == (3, 3):
        ten.reshape((1, 3, 3))
    
    points = ten.shape[0]
                    

    if Q.shape == (3, 3):
        a = np.zeros(ten.shape)
        a[:] = Q       
        Q = a
        
    tenPrime = np.zeros(ten.shape)    
    for i in np.arange(points):
        tenPrime[i] = np.dot(Q[i], np.dot(ten[i], Q[i].T))
    
    return tenPrime



def findOOP(deltaStress, dev, densityRock, z):
    QPrime = plungeTrendToUnitVec(plunge, trend)
    QPrimeInv = QPrime.T

    g = 9.81
    hydroStress = hydroStaticStress(densityRock, g, z)

    regStressPrime = hydroStress + dev
    regStress = rotateTensFull(regStressPrime, QPrimeInv)

    totalStress = regStress + deltaStress

    eigVal, eigVec = np.linalg.eig(totalStress)
    for i in np.arange(eigVec.shape[0]): # it gives eigVec in the wrong order for me. 
        eigVec[i] = eigVec[i].T

    # Sort, from least to greatest (because most compression gives most negative number, this is actually sorting principal stresses from least to greatest)
    for i in np.arange(eigVec.shape[0]):
        sort = np.argsort(eigVal[i])
        eigVal[i] = eigVal[i][sort]
        eigVec[i] = eigVec[i][sort]

    failAng = np.arctan(1/coefFrict) / 2 # from king
    if len(failAng.shape) != 2:
        failAng = np.array(failAng * np.ones(eigVec.shape[0]))

    # I designate normal in direction failAng + 90. Don't change! Later, I check if this actually faces hanging wall and then correct it

    oop1primen = np.array(
        [np.cos( failAng + np.pi/2), np.zeros(failAng.shape), np.sin( failAng + np.pi/2)] ).T
    oop2primen = np.array(
        [np.cos(-failAng - np.pi/2), np.zeros(failAng.shape), np.sin(-failAng - np.pi/2)] ).T

    oop1primes = np.array([np.cos( failAng), np.zeros(failAng.shape), np.sin( failAng)]).T
    oop2primes = np.array([np.cos(-failAng), np.zeros(failAng.shape), np.sin(-failAng)]).T

    oop1n = np.zeros(oop1primen.shape)
    oop2n = np.zeros(oop2primen.shape)
    oop1s = np.zeros(oop1primes.shape)
    oop2s = np.zeros(oop2primes.shape)

    for i in np.arange(oop1n.shape[0]):
        oop1n[i] = np.dot(eigVec[i].T, oop1primen[i])
        oop2n[i] = np.dot(eigVec[i].T, oop2primen[i])
        oop1s[i] = np.dot(eigVec[i].T, oop1primes[i])
        oop2s[i] = np.dot(eigVec[i].T, oop2primes[i])

    upPrime = np.zeros( (eigVec.shape[0], 3) )
    wall1 = np.zeros(oop1primen.shape[0], dtype = 'bool')
    wall2 = np.zeros(oop2primen.shape[0], dtype = 'bool')
    for i in np.arange(upPrime.shape[0]):
        upPrime[i] = np.dot(eigVec[i], np.array([0, 0, 1]))
        wall1[i] = np.dot(oop1primen[i], upPrime[i]) > 0
        wall2[i] = np.dot(oop2primen[i], upPrime[i]) > 0    

    oop1n[~wall1] *= -1   
    oop1s[~wall1] *= -1
    oop2n[~wall2] *= -1
    oop2s[~wall2] *= -1

    s1, d1 = normalToStrikeDip(oop1n)
    s2, d2 = normalToStrikeDip(oop2n)

    strikeVec1 = np.array([np.sin(s1), np.cos(s1), np.zeros(s1.shape)]).T
    strikeVec2 = np.array([np.sin(s2), np.cos(s2), np.zeros(s2.shape)]).T

    r1 = np.zeros(s1.shape)
    r2 = np.zeros(s2.shape)

    for i in np.arange(r1.shape[0]):
        r1[i] = np.arccos(np.dot(strikeVec1[i], oop1s[i]))
        r2[i] = np.arccos(np.dot(strikeVec2[i], oop2s[i]))

    r1[oop1s[:,2]<0]*=-1
    r2[oop2s[:,2]<0]*=-1
    
    return s1, d1, r1, s2, d2, r2

In [ ]:
# Make grid of points for visualizing coulomb stress

xmin = 13
xmax = 13.9
ymin = 42.05
ymax = 42.7
aspect = (xmin-xmax)/(ymin-ymax)

# 2d Mesh values. Can be made 3d by changing zz and nz
nx=50;ny=50; nz=1

# nx=int(nx * cutGrid); ny=int(ny * cutGrid)

xx = np.linspace(xmin,xmax,nx)
yy = np.linspace(ymin,ymax,ny)
zz = -np.array([5000])
xGrid, yGrid, zGrid = np.meshgrid(xx,yy,zz)
totalPoints=nx*ny*nz
lonGrid=xGrid.reshape(totalPoints)
latGrid=yGrid.reshape(totalPoints)
depthGrid=zGrid.reshape(totalPoints)

stressDeviator = 2e6 * np.array([
    [-1, 0, 0], 
    [0, 0, 0],
    [0, 0, 1]
    ])
densityRock = 2700
plunge = np.array([75.29, 13.89, 4.55]) * np.pi / 180 # Principal stress directions
trend = np.array([160.77, -37.96, 53.1]) * np.pi / 180 # From Chiaraluce 2003? read from Nostro
coefFrict = hom.coefFrict
coefSkempt = hom.coefSkempt

In [ ]:
# Load hypocenter info. These are points to be strained, not source quakes. 
# Also create booleans to include/disclude points from analysis. 
magLimitTesting = None
if magLimitTesting:
    print('change that!')

[latP, lonP, depthP, mLP, radP, ST1P, DIP1P, RK1P, ST2P, DIP2P, RK2P, ttP, slipP
    ] = selectedINGV(magLimitTesting, False, reRun=False,path=variable_folder)

s1P, s2P, d1P, d2P, r1P, r2P = sortRupture(ST1P, ST2P, DIP1P, DIP2P, RK1P, RK2P, 
                                                    130*np.pi/180)

FA = ~ ((s1P== 0) * (s2P==0) * (d1P==0) * (d2P==0) * (r1P==0) * (r2P==0) ) # focal available

mainshockInd = np.where(allPoints.mL==np.amax(allPoints.mL) )[0]
aftershock = np.ones(allPoints.mL.size, dtype = bool)
for i in range(aftershock.size):
    if i <= mainshockInd:
        aftershock[i] = False

In [ ]:
# options. 0: run and save, 1: load and don't save, 2: run and don't save, 3: don't run but save, 4: nothing?
rssurf  = 1
rssurfTD= 1
rsheter = 1
rscir   = 1

simulate = aftershock.copy()

# Find stress tensors: (s: stress tensor, g or p: grid or point, cir or het or surf: model)
# Below, I Load, run, save, or whatever was decided. 

#3d gps okada
if (rssurf==0) or (rssurf==2):
    spsurf = np.zeros( (latP.size, 3, 3) )
    spsurf[simulate] = surfF.findStressTen(latP[simulate], lonP[simulate], depthP[simulate])
    sgsurf = surfF.findStressTen(latGrid, lonGrid, depthGrid)
    
if (rssurf==0) or (rssurf==3):
    saveArrays([spsurf, sgsurf],   'stressTenSurf',  variable_folder)
    
if (rssurf==1):
    spsurf, sgsurf   =loadArrays('stressTenSurf',  variable_folder)

#planar gps
if (rsheter==0) or (rsheter==2):
    spHeter = np.zeros( (latP.size, 3, 3) )
    spHeter[simulate] = heterF.findStressTen(latP[simulate], lonP[simulate], depthP[simulate])
    sgHeter = heterF.findStressTen(latGrid, lonGrid, depthGrid)

if (rsheter==0) or (rsheter==3):
    saveArrays([spHeter, sgHeter], 'stressTenHeter', variable_folder)
    
if (rsheter==1):
    spHeter, sgHeter =loadArrays('stressTenHeter', variable_folder)

#cirella
if (rscir==0) or (rscir==2):
    spcir = np.zeros( (latP.size, 3, 3) )
    spcir[simulate] = cir.findStressTen(latP[simulate], lonP[simulate], depthP[simulate])
    sgcir = cir.findStressTen(latGrid, lonGrid, depthGrid)

if (rscir==0) or (rscir==3):
    saveArrays([spcir, sgcir],     'stressTenCir',   variable_folder)
    
if (rscir==1):
    spcir, sgcir     =loadArrays('stressTenCir',   variable_folder)

#3d gps TD
if (rssurfTD==0) or (rssurfTD==2):
    spsurfTD = np.zeros( (latP.size, 3, 3) )
    spsurfTD[simulate] = surfTDF.findStressTenTD(latP[simulate], lonP[simulate], depthP[simulate])
    sgsurfTD = surfTDF.findStressTen(latGrid, lonGrid, depthGrid)
    
if (rssurfTD==0) or (rssurfTD==3):
    saveArrays([spsurfTD, sgsurfTD],   'stressTenSurfTD',  variable_folder)
    
if (rssurfTD==1):
    spsurfTD, sgsurfTD   =loadArrays('stressTenSurfTD',  variable_folder)

    
# Below, I CAN combine stress made from each mainshock model
# with stress made from smaller earthquakes. 
# by default, this isn't done. 
# It should only be done to make Figure S3, or to play around. 
IncludeStressFromSmallQuakes = False # this is all that has to be changed to include small earthquakes as sources
if IncludeStressFromSmallQuakes:
    print('If this cell is ran more than once, we have to reload each models stress tensors!')

    citarealeQuake = allPoints.mL > 3
    citarealeQuake *= allPoints.time > np.datetime64('2009-06-24T00:00:00')
    citarealeQuake *= allPoints.time < np.datetime64('2009-06-26T00:00:00')
    print(allPoints.time[citarealeQuake], '\n',
    allPoints.mL[citarealeQuake])

    minMagSource = 4 # saved with minMagSource = 4 on 04/04/2019
    sources = np.where( # find indecies of events that are sources. Used in naming files.
        np.logical_and(
            np.logical_or(allPoints.mL>=minMagSource, 
                      citarealeQuake), # citraelle quake?
        allPoints.mL!=np.amax(allPoints.mL)
                        )
        )[0]
    smallDCSFolder = 'dcs_small_sources/'
    plt.scatter(allPoints.x[sources], allPoints.y[sources])
    plt.show()

    print(np.sum(allHypo.mL>minMagSource), 'sources \n')
    print('slip of sources', allPoints.slip[sources])

    # options. 0: run and save, 1: load and don't save, 2: run and don't save, 3: don't run but save
    rssmallSource  = 1
    ####### move this ^

    stressTenPsmall = np.zeros( (sources.size, latP.size, 3, 3) )
    stressTenGsmall = np.zeros( (sources.size, latGrid.size, 3, 3))

    for index in sources:
    # Make INGV instance of mainshock model
        smallSource = mainModels('A smallish source quake') # replaced in each loop iteration
        smallSource.mechanicalConstants()

        # Load INGV inverted mainshock: IM
        [smallSource.lat, smallSource.lon, smallSource.depth, smallSource.mL, 
             smallSource.rad, 
             s1, d1, r1, s2, d2, r2, 
             smallSource.tt, smallSource.slip
            ] = [allPoints.lat [[index]], 
                 allPoints.lon [[index]], 
                 allPoints.z   [[index]], 
                 allPoints.mL  [[index]], 
                 allPoints.rad [[index]], 
                 allPoints.st1 [[index]],
                 allPoints.dp1 [[index]],
                 allPoints.rk1 [[index]],
                 allPoints.st2 [[index]],
                 allPoints.dp2 [[index]],
                 allPoints.rk2 [[index]],
                 allPoints.time[[index]], 
                 allPoints.slip[[index]]
            ]

        smallSource.s, __, smallSource.d, __, smallSource.r, __ = sortRupture(
            s1, s2, d1, d2, r1, r2, 130*np.pi/180 )

        smallSource.strikeLength = 2*smallSource.rad
        smallSource.dipLength = 2*smallSource.rad

        smallSource.ss = smallSource.slip * np.cos(smallSource.r)
        smallSource.ds = smallSource.slip * np.sin(smallSource.r)

        # 2.5 cells per 1000 m, but no less than 3 cells, no more than 9 cells
        cellsLine = int(np.ceil(smallSource.rad * 2.5/1000) )
        if cellsLine <3:
            cellsLine = 3
        if cellsLine >9:
            cellsLine = 9

        smallSource.strikeRects =  np.array([cellsLine])
        smallSource.verticalRects = np.array([cellsLine])
        smallSource.makeLists()

        plt.scatter(smallSource.lonL, smallSource.latL, 
                    c = smallSource.dsL,
                    vmin = -.5, vmax = 0)

        simulate = ttP > smallSource.tt

        if (rssmallSource==0) or (rssmallSource==2):
            spsmallSource1 = np.zeros( (latP.size, 3, 3) )
            spsmallSource1[simulate] = smallSource.findStressTen(latP[simulate], lonP[simulate], depthP[simulate])
            sgsmallSource1 = smallSource.findStressTen(latGrid, lonGrid, depthGrid) 

            ni = np.where(sources==index)
            stressTenPsmall[ni] = spsmallSource1
            stressTenGsmall[ni] = sgsmallSource1

    plt.show()

    spsmallSource = np.zeros((latP.size, 3, 3))
    sgsmallSource = np.zeros((latGrid.size, 3, 3))
    for i in np.arange(stressTenPsmall.shape[0]):
        spsmallSource[:] += stressTenPsmall[i]
        sgsmallSource[:] += stressTenGsmall[i]



    if (rssmallSource==0) or (rssmallSource==3):
        saveArrays([spsmallSource, sgsmallSource], 'stressTensmallSource',   variable_folder)

    if (rssmallSource==1):
        spsmallSource, sgsmallSource =loadArrays('stressTensmallSource',   variable_folder)


    
    spsurfTD += spsmallSource
    sgsurfTD += sgsmallSource
    
    spsurf += spsmallSource
    sgsurf += sgsmallSource
    
    spHeter += spsmallSource
    sgHeter += sgsmallSource
    
    spcir += spsmallSource
    sgcir += sgsmallSource

In [ ]:
# Get OOPs
sOOPstdp, dOOPstdp, rOOPstdp, __, __, __ = findOOP(spsurfTD, stressDeviator, densityRock, depthP)
sOOPstdg, dOOPstdg, rOOPstdg, __, __, __ = findOOP(sgsurfTD, stressDeviator, densityRock, depthGrid)

sOOPsp, dOOPsp, rOOPsp, __, __, __ = findOOP(spsurf, stressDeviator, densityRock, depthP)
sOOPsg, dOOPsg, rOOPsg, __, __, __ = findOOP(sgsurf, stressDeviator, densityRock, depthGrid)

sOOPhp, dOOPhp, rOOPhp, __, __, __ = findOOP(spHeter , stressDeviator, densityRock, depthP)
sOOPhg, dOOPhg, rOOPhg, __, __, __ = findOOP(sgHeter , stressDeviator, densityRock, depthGrid)

sOOPcp, dOOPcp, rOOPcp, __, __, __ = findOOP(spcir , stressDeviator, densityRock, depthP)
sOOPcg, dOOPcg, rOOPcg, __, __, __ = findOOP(sgcir , stressDeviator, densityRock, depthGrid)
    

# cs
dCS1surfTD = coulombStress(spsurfTD[FA], 
                         coefFrict, coefSkempt, s=s1P[FA],d=d1P[FA],r=r1P[FA])[2]
dCS2surfTD = coulombStress(spsurfTD[FA], 
                         coefFrict, coefSkempt, s=s2P[FA],d=d2P[FA],r=r2P[FA])[2]
dCSMsurfTD = coulombStress(spsurfTD, coefFrict, coefSkempt, s=sOOPstdp,d=dOOPstdp,r=rOOPstdp)[2]
dCSGsurfTD = coulombStress(sgsurfTD, coefFrict, coefSkempt, s=sOOPstdg,d=dOOPstdg,r=rOOPstdg)[2]


dCS1surf = coulombStress(spsurf[FA], 
                         coefFrict, coefSkempt, s=s1P[FA],d=d1P[FA],r=r1P[FA])[2]
dCS2surf = coulombStress(spsurf[FA], 
                         coefFrict, coefSkempt, s=s2P[FA],d=d2P[FA],r=r2P[FA])[2]
dCSMsurf = coulombStress(spsurf, coefFrict, coefSkempt, s=sOOPsp,d=dOOPsp,r=rOOPsp)[2]
dCSGsurf = coulombStress(sgsurf, coefFrict, coefSkempt, s=sOOPsg,d=dOOPsg,r=rOOPsg)[2]


dCS1Heter  = coulombStress(spHeter[FA], 
                         coefFrict, coefSkempt, s=s1P[FA],d=d1P[FA],r=r1P[FA])[2]
dCS2Heter  = coulombStress(spHeter[FA], 
                         coefFrict, coefSkempt, s=s2P[FA],d=d2P[FA],r=r2P[FA])[2]
dCSMHeter  = coulombStress(spHeter , coefFrict, coefSkempt, s=sOOPhp,d=dOOPhp,r=rOOPhp)[2]
dCSGHeter  = coulombStress(sgHeter , coefFrict, coefSkempt, s=sOOPhg,d=dOOPhg,r=rOOPhg)[2]


dCS1cir  = coulombStress(spcir[FA], 
                         coefFrict, coefSkempt, s=s1P[FA],d=d1P[FA],r=r1P[FA])[2]
dCS2cir  = coulombStress(spcir[FA], 
                         coefFrict, coefSkempt, s=s2P[FA],d=d2P[FA],r=r2P[FA])[2]
dCSMcir  = coulombStress(spcir , coefFrict, coefSkempt, s=sOOPcp,d=dOOPcp,r=rOOPcp)[2]
dCSGcir  = coulombStress(sgcir , coefFrict, coefSkempt, s=sOOPcg,d=dOOPcg,r=rOOPcg)[2]

dCSGsurfTD  = dCSGsurfTD.reshape((nx, ny))
dCSGsurf  = dCSGsurf.reshape((nx, ny))
dCSGHeter = dCSGHeter.reshape((nx, ny))
dCSGcir   = dCSGcir.reshape((nx, ny))

## 5.2 Results

Figure 9 shows the distribution of ∆CFF in map view projected on both OOPs and focal mechanism data which is modelled from our 2D and 3D GPS based mainshock models as well as the planar joint model (Cirella et al., 2013). Figure 10 shows the relationship between slip distributions, aftershocks locations, and ∆CFF. We observe that the distribution of ∆CFF is very complex near the mainshock and is very smooth at large distances from the mainshock. Far from the mainshock, ∆CFF varies minimally between mainshock models, while ∆CFF near the mainshock is strongly dependent on mainshock model parameters. This is also shown in Figure 11. When aftershocks that are part of the L’Aquila fault cluster are not counted, meaning that we are looking only at aftershocks distant from the mainshock, there is increased similarity between models regarding the percent of aftershocks with positive ∆CFF.

When accounting for all aftershocks which had focal mechanisms, the jointly inverted planar mainshock model predicts that more events have positive ∆CFF than do our GPS inverted mainshock models (Figure 11). This is true regardless of whether we used the actual focal mechanism data or OOPs. Our 3D mainshock model predicts the least events to have positive ∆CFF on focal mechanism data, but our planar GPS inverted model finds the least events to have positive ∆CFF on OOPs. For each model, the percent of events with positive ∆CFF increases when we exclude events which were part of the L’Aquila fault cluster.

Figure 12a shows ∆CFF results at different time intervals. This overlays a scatter plot of the magnitude of aftershocks through time. After the mainshock, the frequency of events as well as their magnitudes generally decreases through time until shortly after the MW 4.4 Campotosto earthquake in June which appears to initiate more aftershocks. ∆CFF results are fairly constant through time, but when weighted by $10^M$, the trend emerges that there is very positive ∆CFF immediately after the mainshock. ∆CFF then decays through time until August-September when the percent of events experiencing positive ∆CFF increases again. Figure 12b excludes events that are not part of the L’Aquila cluster. Temporal ∆CFF patterns are similar as in Figure 12a, but these patterns are not as unanimously demonstrated by the three mainshock models, and these patterns are less consistent. This is likely due to the difficulties in modelling near-field stresses.


#### Figure S2
Figure S2. This shows the strike and dip of OOPs. Orientations resulting from our 3D surface using Okada cells and triangular dislocations are both shown and are very similar, suggesting that Okada cells are generally sufficient for modelling relatively simple 3D surfaces. For all models, the orientation of OOPs is dominated by regional stress at great distances from the mainshock. Close to the mainshock, orientations are influenced more by the mainshock.

In [ ]:
# directions of OOPs at 5km depth
yOOPFig, xOOPFig = ChangeAxis(np.amin(latGrid), np.amin(lonGrid), latGrid, lonGrid)
xyvalid = (xOOPFig < 60e3) * (xOOPFig > 20e3) * (yOOPFig < 50e3) * (yOOPFig > 10e3)
xOOPFig = xOOPFig[xyvalid]
yOOPFig = yOOPFig[xyvalid]
xOOPFig -= xOOPFig.min()
yOOPFig -= yOOPFig.min()

OOPList = [sOOPstdg, sOOPsg,sOOPhg,sOOPcg]
OOPStresses = [dCSGsurfTD, dCSGsurf, dCSGHeter, dCSGcir]
OOPLabelList = ['3D GPS TD', '3D GPS Okada', '2D GPS', '2D Joint']
fig, ax = plt.subplots(4)
fig.set_size_inches(7, 7*4)
for i, sMod in enumerate(OOPList):

    dx = np.sin(sMod)[xyvalid]
    dy = np.cos(sMod)[xyvalid]
#     ax.set_xlim(np.amin(xGrid), np.amax(xGrid))
#     ax.set_ylim(np.amin(yGrid), np.amax(yGrid))
#     ax.imshow(OOPStresses[i], zorder=2, origin = 'lower')
    ax[i].quiver(
        xOOPFig/1e3, yOOPFig/1e3,
        dx, dy,
        headwidth = 0, scale = 30, pivot = 'middle')
    dxd = np.sin(sMod+np.pi/2)[xyvalid]
    dyd = np.cos(sMod+np.pi/2)[xyvalid]
    ax[i].quiver(
        xOOPFig/1e3, yOOPFig/1e3,
        dxd, dyd, pivot = 'tail',
        headwidth = 0, scale = 50)
    ax[i].set_title(OOPLabelList[i])
    plt.xlabel('East (km)')
    plt.ylabel('North (km)')
    plt.grid()
plt.tight_layout()
plt.show()
    # sOOPsg, dOOPsg, rOOPsg

#### Figure 9
Figure 9. ∆CFF on aftershocks resulting from different mainshock models. On the left, all 51011 aftershocks were used assuming OOPs. On the right, we used 3415 actual focal mechanisms of aftershocks. We determined whether both, one, or neither plane experienced positive ∆CFF induced from the mainshock. The color of aftershocks is such that red means positive ∆CFF on both focal planes and blue means negative ∆CFF on both focal planes. Yellow means positive ∆CFF on one but not the other focal plane. For the stress image in the background of these figures, ∆CFF is calculated on a grid of OOPs at 5 km depth.

In [ ]:
# Make array that shows us which are aftershocks
# Used for all CFF analysis
# mainshockInd = np.where(allPoints.mL==np.amax(allPoints.mL) )[0]
# aftershock = np.ones(allPoints.mL.size, dtype = bool)
# for i in range(aftershock.size):
#     if i <= mainshockInd:
#         aftershock[i] = False
aftershockFocal = np.logical_and(aftershock, FA)

boo = db2.labels==0 # this IS part of main cluster. 

allPoints = hypoData()
allPoints.minMag = None
allPoints.getHypocenters()
focalBool = (FA)[boo] # sub of focal available

# Make boolan arrays which show which hypocenters to consider
includeSurf = np.logical_and(
    ~boo, aftershock)[FA] # NOT part of surface, is an aftershock
includeAll = aftershock[FA] # subset with focal mechanisms and aftershock

includeSurfFull = np.logical_and( # Full means matrix is size of all points, not just ones with focal mechanisms
    ~boo, aftershock*FA ) 
includeAllFull = aftershock*FA 



In [ ]:
partsurf = db2.labels==0 # part of surface
aft = aftershock # aftershock
foc = FA # focal available


In [ ]:
# Figure 9

# Make individual scatter and image plots for figure below
def dCS_sca_im(ax, high, low, lat, lon, dCS1, dCS2=None,
                 dCSG = None, strainers=None, maxIm = .1, minIm = -.1,
              aspect = 4/3, alpha = .9, errBarWidth = 10000):
    if dCSG is not None:
        im = ax.imshow(dCSG*1e-6, extent = (xmin, xmax, ymin, ymax), 
               origin = 'lower',interpolation = 'bilinear', zorder=0,
               cmap = 'seismic', vmax = maxIm, vmin = minIm, 
                       aspect = aspect, 
                       alpha=alpha)
        
        barX = 1/7*(xmax-xmin)+xmin
        barY = 1/10*(ymax-ymin)+ymin
        __, lonNew = xyToLonLat(barY, barX, 10000, 0)
        dxErrBar = (lonNew-barX)*.5 # Because err bar goes both directions
        bar = ax.errorbar(barX, barY, xerr = dxErrBar,
                         elinewidth=2, color = 'white', capsize=4)
        ax.text(barX, barY+.5*dxErrBar, '10 km', color = 'white', 
                horizontalalignment = 'center')

    if dCS2 is None:
        dCS2 = dCS1.copy()
    
    color, legend_markers, labels = greenRed(dCS1, dCS2, high, low)
    
    scatterP = ax.scatter(lon, lat, c = color, s = 10, zorder=1,
        linewidths=.5,edgecolors='black' )
    
    if strainers is not None:
        scatterF = ax.scatter(lon[strainers], lat[strainers], c = color, s = 500, 
            marker = '*', zorder = 2)

    return im, scatterP, bar


minStress=-.1 # MPa, for background stress image
maxStress=.1

# mainshock plane, plane1+2, grid
plots = 3

# Put all stresses into one array for a for loop
dCSCombine = np.array([
    [dCSMcir, dCS1cir, dCS2cir, dCSGcir], 
    [dCSMHeter, dCS1Heter, dCS2Heter, dCSGHeter],
    [dCSMsurfTD, dCS1surfTD, dCS2surfTD, dCSGsurfTD]
#    ,[dCSMsurfFine, dCS1surfFine, dCS2surfFine, dCSGsurfFine]
])



# Silly way of also putting these booleans in an array
inclusion = np.array([
    aftershock[foc],
    aftershock[foc],
    aftershock[foc]
#    ,aftershock[foc]
])

inBounds = ~((latP > ymax) + (latP < ymin) + (lonP > xmax) + (lonP<xmin)).astype('bool')
inclusionFull = np.array([
    aftershock * inBounds,
    aftershock * inBounds,
    aftershock * inBounds
#    ,aftershock * inBounds
])

fig, axes = plt.subplots(nrows=dCSCombine.shape[0], ncols=2, figsize=(12, 24))
matplotlib.rcParams.update({'font.size': 12})

# These aren't relevant but are stuck as part of my code
high = 0
low = 0

for i in np.arange(plots):
    im, scatter1, bar1 = dCS_sca_im(axes[i,0], high, low, latP[inclusionFull[i]], lonP[inclusionFull[i]], 
               dCSCombine[i][0][inclusionFull[i]], dCSCombine[i][0][inclusionFull[i]], dCSCombine[i, 3],
                             minIm = minStress, maxIm = maxStress, aspect = aspect)
    
    im, scatter2, bar2 = dCS_sca_im(axes[i,1], high, low, latP[FA][inclusion[i]], lonP[FA][inclusion[i]], 
               dCSCombine[i][1][inclusion[i]], dCSCombine[i][2][inclusion[i]], dCSCombine[i, 3],
                             minIm = minStress, maxIm = maxStress, aspect = aspect)
    
# Label slip distributions
axes[0,0].set_ylabel('Planar Joint')
axes[1,0].set_ylabel('Planar GPS')
axes[2,0].set_ylabel('3D GPS')

for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        axes[i, j].set_yticklabels(['{:.1f}$\degree$'.format(x) for x in plt.gca().get_yticks()])
        axes[i, j].set_xticklabels(['{:.1f}$\degree$'.format(x) for x in plt.gca().get_xticks()])


# Label planes
axes[0,0].set_title('OOPs')
axes[0,1].set_title('Actual Focal Mechanisms')
plt.suptitle('$\Delta$CFF From Models', y = .885)

# Colorbar
widthC = .35
tallC = 0.01
cbar_ax = fig.add_axes([.5 - .5 * widthC, 
                        .1 - .5 * tallC, 
                        widthC, tallC])
cbar = fig.colorbar(im, cax=cbar_ax, orientation = 'horizontal')
cbar.set_label('$\Delta$CFF (MPa)')
cticks = np.linspace(minStress, maxStress, 5)
cbar.set_ticks(cticks)

plt.show()

#### Figure 10
Figure 10. This shows the slip distribution of models compared to the location of aftershocks and their calculated ∆CFF. The slip distributions are the same as in Figure 8, while the color scheme for aftershocks is the same as in Figure 9. (a) Slip distribution jointly inverted by Cirella et al. (2012), (b) slip distribution on our inverted 2D fault plane (c) slip distribution inverted with our 3D fault.

In [ ]:
def plotly_trisurf2(x, y, z, simplices, colormap=cm.RdBu, plot_edges=None, colorsSurf=None):
    #x, y, z are lists of coordinates of the triangle vertices 
    #simplices are the simplices that define the triangularization;
    #simplices  is a numpy array of shape (no_triangles, 3)
    #insert here the  type check for input data

    points3D=np.vstack((x,y,z)).T
    tri_vertices=map(lambda index: points3D[index], simplices)# vertices of the surface triangles  
    zmean=[np.mean(tri[:,2]) for tri in tri_vertices ]# mean values of z-coordinates of 
                                                      #triangle vertices
    min_zmean=np.min(zmean)
    max_zmean=np.max(zmean)
    facecolor=[map_z2color(zz,  colormap, min_zmean, max_zmean) for  zz in zmean]
    

    I,J,K=tri_indices(simplices)
    
    slpmean = 1/3 * (colorsSurf[I]+colorsSurf[J]+colorsSurf[K])
    min_slpmean = np.amin(slpmean)
    max_slpmean = np.amax(slpmean)
    facecolor=[map_z2color(slp,  colormap, min_slpmean, max_slpmean) for  slp in slpmean]

    triangles=go.Mesh3d(x=x,
                     y=y,
                     z=z,
                     facecolor=facecolor,
                     opacity=.7,
                     i=I,
                     j=J,
                     k=K
                    )

    if plot_edges is None:# the triangle sides are not plotted 
        return [triangles]
    else:
        #define the lists Xe, Ye, Ze, of x, y, resp z coordinates of edge end points for each triangle
        #None separates data corresponding to two consecutive triangles
        lists_coord=[[[T[k%3][c] for k in range(4)]+[ None]   for T in tri_vertices]  for c in range(3)]
        Xe, Ye, Ze=[reduce(lambda x,y: x+y, lists_coord[k]) for k in range(3)]

        #define the lines to be plotted
        lines=go.Scatter3d(x=Xe,
                        y=Ye,
                        z=Ze,
                        mode='lines',
                        line=dict(color= 'rgb(50,50,50)', width=1.5)
               )
        return [triangles, lines]

In [ ]:
def stresses3D(xp, yp, zp, xs=None, ys=None, zs=None, colors=None, colorsSurf=None,
              pointSize=.5, pLineWidthR=1/4, save = False, saveName = 'delete'):
    plots = []
    ###### scatter plot
    pointSize=pointSize
    scatterPlot = [go.Scatter3d(
        x=xp, y=yp, z=zp,
        mode='markers',
        marker=dict(
            size=pointSize,
            color=colors, # change
            line=dict(
                    color='rgb(0,0,0)',
                    width=pLineWidthR * pointSize
                ),
            opacity=1) )]
    plots = plots + scatterPlot
    ######    

    if xs is not None:
        ###### Triangulation surface
        points2D=np.vstack([xs,ys]).T
        tri=Delaunay(points2D)

        surfacePlot=plotly_trisurf2(xs, ys, zs,
                                   tri.simplices, colormap=cm.cubehelix, plot_edges=None,
                                   colorsSurf=colorsSurf)
#         print(surfacePlot)
        plots = plots + surfacePlot
        ######


    ###############
    ############################ 
    textFontDict = {'color':'red', 'size':18}
    ### north arrow
    xR = 10e3
    xNA = np.amax(xR-3e3)
    yNA = 28e3#np.amax(self.y)
    zNA = -14e3#np.amin(self.z)
    yBase = yNA - 5e3

    northLine = go.Scatter3d(
        x=[xNA, xNA], y=[yBase, yNA], z=[zNA, zNA],
        text = np.array(['', 'N']),
        mode='lines+text',#Change?
        line  =dict(width = 5, color = '#7f7f7f'),
        textposition = 'middle left',
        textfont = textFontDict
    )

    northCone = dict(
        type = 'cone',
        colorscale = 'Greys',
        showscale = False,
        x = [xNA], y = [yNA], z = [zNA],
        u = [0], v = [2e3], w = [0]
        )
    ###

    ###  scale bar      
    xN = np.array([xR, xR, xR])
    yN = np.array([-10e3, -5e3, 0])+yNA
    zN = np.array([zNA, zNA, zNA])
    scaleBar = go.Scatter3d(
        x=xN,
        y=yN,
        z=zN,
        text = np.array(['', '10 km', '']),
        mode='lines+text',
        line  =dict(width = 15, color = '#7f7f7f'),
        textposition='middle right',
        textfont = textFontDict
    )

    upLine = go.Scatter3d(
        x=[xNA, xNA], y=[yBase, yBase], z=[zNA, zNA+5e3],
        text = np.array(['', 'UP']),
        mode='lines+text',#Change?
        line  =dict(width = 5, color = '#7f7f7f'),
        textposition = 'middle left',
        textfont = textFontDict
    )

    upCone = dict(
        type = 'cone',
        colorscale = 'Greys',
        showscale = False,
        x = [xNA], y = [yBase], z = [zNA+5e3],
        u = [0], v = [0], w = [2e3]
        )
    ###
    plots.append(scaleBar)
    plots.append(northLine)
    plots.append(northCone)
    plots.append(upLine)
    plots.append(upCone)
    #########################
    ###############

    layout = go.Layout(
        showlegend=False,
        autosize=False,
        width=1000,
        height=1000,
        scene=dict(
            camera=dict(eye=dict(x=1.75, y=-0.7, z= 0.75) ),
            xaxis = dict(showticklabels=False, title=''),
            yaxis = dict(showticklabels=False, title=''),
            zaxis = dict(showticklabels=False, title='')
                  )  
        )

    fig2 = go.Figure(data=plots, layout=layout)
    if save:
        py.iplot(fig2, filename = saveName)
    else:
        plot(fig2)
        

In [ ]:
model3Dplot = [cir, heterF, surfF, 
               cir, heterF, surfF]
dCS3DFig = np.array([
    [dCS1cir,     dCS2cir    ],
    [dCS1Heter,   dCS2Heter  ],
    [dCS1surfTD,  dCS2surfTD ],
    [dCSMcir,     dCSMcir    ],
    [dCSMHeter,   dCSMHeter  ],
    [dCSMsurfTD,  dCSMsurfTD ]
])
inclAll = np.ones(FA.shape, dtype = 'bool')
incl = np.array([
    FA, FA, FA, inclAll, inclAll, inclAll
])
saveNames = ['Joint Focal',
            '2D GPS Focal',
            '3D GPS focal',
            'Joint OOP',
            '2D GPS OOP',
            '3D GPS OOP']

include3DPlot = [0, 0, 0, 0, 0, 0]
pointsizes = [2, 2, 2, 1, 1, 1]
if runAllFigures:
    for i in range(3):
        include3DPlot[i] = 1
print('to see OOP relationship, change include3DPlot[3:] to 1')

putOnline = False

for i, mod in enumerate(model3Dplot):
    if include3DPlot[i]:
        y3D, x3D = ChangeAxis(allPoints.latCent, allPoints.lonCent, mod.latL[0], mod.lonL[0])

        colors = np.zeros(np.sum(incl[i]), dtype = 'object')
        stressFun1 = dCS3DFig[i][0]
        stressFun2 = dCS3DFig[i][1]

        both = (stressFun1>0)*(stressFun2>0)
        justone    = np.logical_or((stressFun1>0),(stressFun2>0))
        neither = ~ ( np.logical_or(justone, both) )

        colors[justone]='yellow'
        colors[both] = 'red'
        colors[neither]='blue'

        y3D, x3D = ChangeAxis(allPoints.latCent, allPoints.lonCent, mod.latL[0], mod.lonL[0])
        slp = np.sqrt( mod.ssL[0]**2 + mod.dsL[0]**2 )

        stresses3D(allPoints.x[incl[i]], allPoints.y[incl[i]], allPoints.z[incl[i]], 
                   x3D, y3D, mod.depthL[0],
                   colors=colors,
                   colorsSurf = slp,
                  pointSize=pointsizes[i], saveName = saveNames[i], save=putOnline)
    else:
        pass

#### Figure 11
Figure 11. ∆CFF resulting from different mainshock models. Red shows the percent of aftershocks which experienced +∆CFF on both focal mechanism planes, yellow for one and not the other focal mechanism plane, and blue for neither focal mechanism plane. The OOP results (left) include all 51011 aftershocks. The other results (center and right) use the real focal mechanism solutions on 3415 aftershocks. (a) Shows ∆CFF results on all aftershocks, regardless of their location. Comparing our GPS based planar and 3D models, the 3D surface finds more OOPs with +∆CFF, while the planar GPS based model find more actual focal mechanisms with +∆CFF. (b) Shows results when aftershocks that are part of the mainshock fault cluster are excluded. (c) Shows results when aftershocks which are not part of the mainshock fault cluster are excluded.

In [ ]:
def histograms(title, otherBool=None):
    subplots = 3
    fig, ax = plt.subplots(1, subplots)
    fig.set_size_inches(11, 5)

    labelsbar = ('Planar\nJoint','Planar\nGPS','3D\nGPS'
                 )
    labelsaxes = ('OOPs', 'Actual Focal Mechanisms', 'Weighted $10^{M}$')

    dCSHist = np.array([
        [dCS1cir,   dCS2cir],
        [dCS1Heter, dCS2Heter], 
#         [dCS1surf,  dCS2surf]
        [dCS1surfTD, dCS2surfTD] # Checking fineness of surface
    ])
    dCSMHist = np.array([
        [dCSMcir,   dCSMcir],
        [dCSMHeter, dCSMHeter], 
#         [dCSMsurf,  dCSMsurf]
        [dCSMsurfTD, dCSMsurfTD] # Checking fineness of surface
    ])
    dCSHistAll = [dCSMHist, dCSHist, dCSHist
                 ,dCSMHist # Checking fineness of surface
                 ]

    if otherBool is None:
        otherBool = np.ones(aftershock.size, dtype = 'bool')
    
    inclusionSub = aftershock * otherBool
        
    inclusion = np.array([
        (inclusionSub)[foc],
        (inclusionSub)[foc],
#         (inclusionSub)[foc]
        (inclusionSub)[foc] # Checking fineness of surface
    ])
    inclusionM = np.array([
        (inclusionSub),
        (inclusionSub),
#         (inclusionSub)
        (inclusionSub) # Checking fineness of surface
    ])
    inclusionAll = [inclusionM, inclusion, inclusion]

    weights = [None, None, 10**allPoints.mL[foc] ]

    ###
    for axes in range(subplots):
        bars = dCSHistAll[axes].shape[0]
        p0 = np.zeros(bars); p1 = np.zeros(bars); p2 = np.zeros(bars)
        ind = np.arange(bars)
        width = 0.45

        for i in np.arange(bars):
            stress1 = dCSHistAll[axes][i][0]
            stress2 = dCSHistAll[axes][i][1] 
            inc = inclusionAll[axes][i]
            wet = weights[axes]
            p0[i], p1[i], p2[i] = p012(stress1, stress2, inclusion = inc, weights = wet)

            shiftUp = 1
            horizAlign = 'center'
            ax[axes].text(ind[i], p2[i]+shiftUp, '{:04.1f}'.format(p2[i]),
                          horizontalalignment = horizAlign, color = 'black')
            ax[axes].text(ind[i], p1[i]+p2[i]+shiftUp, '{:04.1f}'.format(p2[i]+p1[i]),
                          horizontalalignment = horizAlign, color = 'black')

        im2 = ax[axes].bar(ind, p2, width, color = 'red')
        im1 = ax[axes].bar(ind, p1, width, bottom=p2, color = 'yellow')
        im0 = ax[axes].bar(ind, p0, width, bottom = p2+p1, color='blue')
        ax[axes].set_xticks(ind)
        ax[axes].set_xticklabels(labelsbar)
        ax[axes].set_xlabel(labelsaxes[axes], labelpad = 10)


        ###

    ax[0].set_ylabel('Percent psitive')
    plt.tight_layout(pad = 1.1)
    for i in range(subplots):
        ax[i].set_yticklabels(['{:.0f}%'.format(x) for x in plt.gca().get_yticks()])



    plt.suptitle(title, y = 1.05)
    plt.show()
    

In [ ]:
matplotlib.rcParams.update({'font.size': 15})
histograms('$\Delta CFF$ from Models on Aftershocks')
histograms('$\Delta CFF$ only where Focal Mechanisms are Available', otherBool = FA)
histograms('$\Delta CFF$ in L\'Aquila cluster', otherBool = partsurf)
histograms('$\Delta CFF$ not in L\'Aquila cluster', otherBool = ~partsurf)

In [ ]:
def histograms2(title, otherBool=None):
    subplots = 3
    fig, ax = plt.subplots(1, subplots)
    fig.set_size_inches(12, 5.5)

    labelsbar = ('Okada','TD'
                 )
    labelsaxes = ('OOPs', 'Actual Focal Mechanisms', 'Weighted $10^{M}$')

    dCSHist = np.array([
#         [dCS1cir,   dCS2cir],
#         [dCS1Heter, dCS2Heter], 
        [dCS1surf,  dCS2surf],
        [dCS1surfTD, dCS2surfTD] # Checking fineness of surface
    ])
    dCSMHist = np.array([
#         [dCSMcir,   dCSMcir],
#         [dCSMHeter, dCSMHeter], 
        [dCSMsurf,  dCSMsurf],
        [dCSMsurfTD, dCSMsurfTD] # Checking fineness of surface
    ])
    dCSHistAll = [dCSMHist, dCSHist, dCSHist
                 ,dCSMHist # Checking fineness of surface
                 ]

    if otherBool is None:
        otherBool = np.ones(aftershock.size, dtype = 'bool')
    
    inclusionSub = aftershock * otherBool
        
    inclusion = np.array([
        (inclusionSub)[foc],
#         (inclusionSub)[foc],
#         (inclusionSub)[foc]
        (inclusionSub)[foc] # Checking fineness of surface
    ])
    inclusionM = np.array([
        (inclusionSub),
#         (inclusionSub),
#         (inclusionSub)
        (inclusionSub) # Checking fineness of surface
    ])
    inclusionAll = [inclusionM, inclusion, inclusion]

    weights = [None, None, 10**allPoints.mL[foc] ]

    ###
    for axes in range(subplots):
        bars = dCSHistAll[axes].shape[0]
        p0 = np.zeros(bars); p1 = np.zeros(bars); p2 = np.zeros(bars)
        ind = np.arange(bars)
        width = 0.45

        for i in np.arange(bars):
            stress1 = dCSHistAll[axes][i][0]
            stress2 = dCSHistAll[axes][i][1] 
            inc = inclusionAll[axes][i]
            wet = weights[axes]
            p0[i], p1[i], p2[i] = p012(stress1, stress2, inclusion = inc, weights = wet)

            shiftUp = 1
            horizAlign = 'center'
            ax[axes].text(ind[i], p2[i]+shiftUp, '{:04.1f}'.format(p2[i]),
                          horizontalalignment = horizAlign, color = 'black')
            ax[axes].text(ind[i], p1[i]+p2[i]+shiftUp, '{:04.1f}'.format(p2[i]+p1[i]),
                          horizontalalignment = horizAlign, color = 'black')

        im2 = ax[axes].bar(ind, p2, width, color = 'red')
        im1 = ax[axes].bar(ind, p1, width, bottom=p2, color = 'yellow')
        im0 = ax[axes].bar(ind, p0, width, bottom = p2+p1, color='blue')
        ax[axes].set_xticks(ind)
        ax[axes].set_xticklabels(labelsbar)
        ax[axes].set_xlabel(labelsaxes[axes], labelpad = 10)


        ###

    ax[0].set_ylabel('Percent psitive')
    plt.tight_layout(pad = 1.1)
    for i in range(subplots):
        ax[i].set_yticklabels(['{:.0f}%'.format(x) for x in plt.gca().get_yticks()])



    plt.suptitle(title, y = 1.05)
    plt.show()
    

In [ ]:
"""this plot is only in the Jupyter Notebook. 
It simply shows that using Okada vs Triangular Dislocations
does not produce importantly different results
for our 3D fault surface"""
histograms2('$\Delta CFF$ Okada vs. Triangular dislocation')

#### Figure 12
Figure 12. This shows how ∆CFF changes through time and how it relates to the magnitude and frequency of earthquakes. Black dots represent ML of all earthquakes. For (a), in order to determine the number of planes from actual focal mechanisms with positive ∆CFF, we used all 3415 events with available focal mechanisms. To determine whether OOPs has positive ∆CFF, all 51011 aftershocks were used. For (b), we used the subset of aftershocks which were part of the L’Aquila cluster. The horizontal axis is time. The number of events with positive ∆CFF is shown unweighted and weighted by $10^M$.


In [ ]:
matplotlib.rcParams.update({'font.size': 17})

# Figure 12
intervals = 9
time = allPoints.time

dcscomb2 = np.array([
    [dCS1cir,     dCS2cir,    dCSMcir   ],  
    [dCS1Heter,   dCS2Heter,  dCSMHeter ],
    [dCS1surfTD,  dCS2surfTD, dCSMsurfTD]
])

linestyles = [':', '--', '-']

wholePlotTitles = np.array(['Unweighted', 'Weighted $10^{M}$'])

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 14))
matplotlib.rcParams.update({'font.size': 12})

timeMin = time[allPoints.mL==np.amax(allPoints.mL)]
timeMax = np.amax(time)
dTime = (timeMax-timeMin)/intervals
timeSlices = timeMin + np.arange(intervals+1) * dTime
timeSlicesPlot = timeSlices[:-1] + .5 * dTime
noFoc = np.datetime64('2009-09-01T00:00:00')
startTime = np.datetime64('2009-03-15T00:00:00')
toEarly = allPoints.time<startTime

p0 = np.zeros(intervals)
p1 = np.zeros(intervals)
p2 = np.zeros(intervals)
p0M= np.zeros(intervals)
p1M= np.zeros(intervals)

for w in [0, 1]:
    for j in range(dcscomb2.shape[0]):
        for i in np.arange(intervals):
            timeBool = (time >= timeSlices[i]) * (time <= timeSlices[i+1])
            timeBool = timeBool * aftershock# * exclmp

            if w == 0:
                weights = np.ones(timeBool.shape)
            elif w == 1:
                weights = 10**allPoints.mL
            stress1 = dcscomb2[j][0]
            stress2 = dcscomb2[j][1]
            stressM = dcscomb2[j][2]
            p0[i], p1[i], p2[i] = p012(stress1, stress2, (timeBool)[FA],
                                       weights = weights[FA])
            p0M[i], __, p1M[i] = p012(stressM, stressM, (timeBool),
                                       weights = weights)
        
        shiftAmt = .045
        num = np.sum(~toEarly)
        shift = (np.random.random(num)-.5) * 2 * shiftAmt
        axes[w].scatter(allPoints.time[~toEarly], allPoints.mL[~toEarly] +shift,#
                    s = 3.5, linewidths=.02, edgecolors = 'k', c = 'k')
        axes[w].scatter(timeMin, 0, s = 0)
        axes[w].set_ylabel('$M_L$', fontsize = 16)

        ax2 = axes[w].twinx()
        incl = timeSlicesPlot<noFoc
        ax2.plot(   timeSlicesPlot, p1M        
                 , c = 'green'  , linestyle = linestyles[j], linewidth=5 ) 
        ax2.plot(   timeSlicesPlot[incl], (p1+p2)[incl] 
                 , c = 'yellow', linestyle = linestyles[j], linewidth=5 )
        ax2.plot(   timeSlicesPlot[incl], (p2)[incl]
                 , c = 'red'   , linestyle = linestyles[j], linewidth=5 )
        ax2.set_ylim(0, 100)
        plt.gca().set_yticklabels(['{:.0f}%'.format(x) for x in plt.gca().get_yticks()])
        ax2.set_title(wholePlotTitles[w])
        ax2.set_ylabel('Percent Positive', fontsize=16)
 

legendFract = (timeSlices[0]-startTime)/3
legendLeft = startTime - legendFract
legendRight = legendLeft
legendTime = np.array([legendLeft, legendRight])
legendMag = np.zeros(2)
legendbar = [axes[0].plot(legendTime, legendMag,
    c = 'k', linestyle = linestyles[i], linewidth=5)[0] for i in range(dcscomb2.shape[0])]
labels = np.array(['Planar Joint', 'Planar GPS', '3D GPS',
                   '+$\Delta$CFF on OOP', 
                   '+$\Delta$CFF one plane', 
                   '+$\Delta$CFF two planes'])
colors = ['green', 'yellow', 'red']
for i in range(3):
    legendbar.append(
    axes[0].plot(legendTime, legendMag,
                c = colors[i], linewidth=5)[0]
    )
# for j in range(dcscomb2.shape[0]):
#     legendbar.append()
# axes[0].legend()
axes[0].legend(legendbar, labels, loc = 7, fontsize=15)
        
plt.suptitle('$\Delta$CFF Through Time', fontsize=20)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
# axes[1].plot(0, 0, c='k', linestyle = linestyles[0], linewidth=5)
plt.show()

In [ ]:
#For only aftershocks in main cluster

matplotlib.rcParams.update({'font.size': 17})

intervals = 9
time = allPoints.time # only where focal mechanisms available

dcscomb2 = np.array([
    [dCS1cir,     dCS2cir,    dCSMcir   ],  
    [dCS1Heter,   dCS2Heter,  dCSMHeter ],
    [dCS1surfTD,  dCS2surfTD, dCSMsurfTD]
])

linestyles = [':', '--', '-']

wholePlotTitles = np.array(['Unweighted', 'Weighted $10^{M}$'])

fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(14, 14))
matplotlib.rcParams.update({'font.size': 12})

timeMin = time[allPoints.mL==np.amax(allPoints.mL)]
timeMax = np.amax(time)
dTime = (timeMax-timeMin)/intervals
timeSlices = timeMin + np.arange(intervals+1) * dTime
timeSlicesPlot = timeSlices[:-1] + .5 * dTime
noFoc = np.datetime64('2009-09-01T00:00:00')
startTime = np.datetime64('2009-03-15T00:00:00')
toEarly = allPoints.time<startTime

p0 = np.zeros(intervals)
p1 = np.zeros(intervals)
p2 = np.zeros(intervals)
p0M= np.zeros(intervals)
p1M= np.zeros(intervals)

for w in [0, 1]:
    for j in range(dcscomb2.shape[0]):
        for i in np.arange(intervals):
            timeBool = (time >= timeSlices[i]) * (time <= timeSlices[i+1])
            timeBool = timeBool * aftershock

            if w == 0:
                weights = np.ones(timeBool.shape)
            elif w == 1:
                weights = 10**allPoints.mL
            stress1 = dcscomb2[j][0]
            stress2 = dcscomb2[j][1]
            stressM = dcscomb2[j][2]
            p0[i], p1[i], p2[i] = p012(stress1, stress2, (partsurf*timeBool)[FA],
                                       weights = (weights)[FA])
            p0M[i], __, p1M[i] = p012(stressM, stressM, partsurf*timeBool,
                                       weights = weights)
        shiftAmt = .045
        num = np.sum((~toEarly)*partsurf)
        shift = (np.random.random(num)-.5) * 2 * shiftAmt
        axes[w].scatter(allPoints.time[(~toEarly)*partsurf],
                        allPoints.mL[(~toEarly)*partsurf] +shift,#
                    s = 3.5, linewidths=.02, edgecolors = 'k', c = 'k')
        axes[w].scatter(timeMin, 0, s = 0)
        axes[w].set_ylabel('$M_L$', fontsize = 16)

        ax2 = axes[w].twinx()
        incl = timeSlicesPlot<noFoc
        ax2.plot(   timeSlicesPlot, p1M        
                 , c = 'green'  , linestyle = linestyles[j], linewidth=5 ) 
        ax2.plot(   timeSlicesPlot[incl], (p1+p2)[incl] 
                 , c = 'yellow', linestyle = linestyles[j], linewidth=5 )
        ax2.plot(   timeSlicesPlot[incl], (p2)[incl]
                 , c = 'red'   , linestyle = linestyles[j], linewidth=5 )
        ax2.set_ylim(0, 100)
        plt.gca().set_yticklabels(['{:.0f}%'.format(x) for x in plt.gca().get_yticks()])
        ax2.set_title(wholePlotTitles[w])
        ax2.set_ylabel('Percent Positive', fontsize=16)
 

legendFract = (timeSlices[0]-startTime)/3
legendLeft = startTime - legendFract
legendRight = legendLeft
legendTime = np.array([legendLeft, legendRight])
legendMag = np.zeros(2)
legendbar = [axes[0].plot(legendTime, legendMag,
    c = 'k', linestyle = linestyles[i], linewidth=5)[0] for i in range(dcscomb2.shape[0])]
labels = np.array(['Planar Joint', 'Planar GPS', '3D GPS',
                   '+$\Delta$CFF on OOP', 
                   '+$\Delta$CFF one plane', 
                   '+$\Delta$CFF two planes'])
colors = ['green', 'yellow', 'red']
for i in range(3):
    legendbar.append(
    axes[0].plot(legendTime, legendMag,
                c = colors[i], linewidth=5)[0]
    )
# for j in range(dcscomb2.shape[0]):
#     legendbar.append()
# axes[0].legend()
axes[0].legend(legendbar, labels, loc = 7, fontsize=15)
        
plt.suptitle('$\Delta$CFF on Aftershocks in L\'Aquila Cluster', fontsize=20)
fig.tight_layout(rect=[0, 0.03, 1, 0.95])
# axes[1].plot(0, 0, c='k', linestyle = linestyles[0], linewidth=5)
plt.show()

#### Figure 13
Figure 13. This compares ∆CFF to the hypocenters minimum distance from the mainshock. For each event, the plotted ∆CFF was chosen from the focal plane where ∆CFF was highest. Colors are the same as in Figure 9. Vertical lines are drawn at integer intervals of a fault cells size (the width of a triangle or Okada cell). Ideally, aftershocks should always be at least one cell size away from the fault.

In [ ]:
matplotlib.rcParams.update({'font.size': 12})


dcscomb1 = np.array([
    [dCS1cir,   dCS2cir, dCSMcir],  
    [dCS1Heter, dCS2Heter, dCSMHeter],
#     [dCS1surf,  dCS2surf, dCSMsurf]
    [dCS1surfTD,dCS2surfTD,dCSMsurfTD]
])

titles = ['Planar Joint', 'Planar GPS', '3D GPS']

otherBool = partsurf
otherBool = np.ones(partsurf.size, dtype = 'bool')
includeHere =  (aftershock*otherBool)[FA]
includeHereFull = (aftershock*otherBool)


stressLim = 1

yPoints, xPoints = ChangeAxis(0, 0, allHypo.lat[FA*aftershock*otherBool], 
                                    allHypo.lon[aftershock*FA*otherBool])
zPoints = allHypo.z[aftershock*FA*otherBool]

fig, ax = plt.subplots(3)
fig.set_size_inches(14, 20)

for i, mod in enumerate([cir, heterF, surfTDF]):
#     plt.figure(figsize=(12,8))
    stress1 = dcscomb1[i][0]
    stress2 = dcscomb1[i][1]
    stressM = dcscomb1[i][2]
    
    s1big = stress1>stress2
    
    stressGre = np.zeros(stress1.shape)
    stressMin = np.zeros(stress1.shape)
    
    stressGre[s1big] = stress1[s1big] 
    stressGre[~s1big]= stress2[~s1big]
    stressMin[~s1big]= stress1[~s1big]
    stressMin[s1big] = stress2[s1big] 
    
    stressMin *= 1e-6
    stressGre *= 1e-6
    
    yFaults, xFaults = ChangeAxis(0, 0, mod.latL[0], mod.lonL[0])
    zFaults = mod.depthL[0]
    modHypDist = distances(xPoints, yPoints, zPoints, 
              xFaults, yFaults, zFaults)
    distPlot = modHypDist.min(axis = 1)
    
    # Decide colors
    pNum = (stressGre>0).astype('int') + (stressMin>0).astype('int')
    colorsMagStr = np.zeros(pNum.shape, dtype = 'str')
    colorsMagStr[pNum == 0] = 'blue'
    colorsMagStr[pNum == 1] = 'yellow'
    colorsMagStr[pNum == 2] = 'red'
    
    ax[i].scatter(distPlot, stressGre[includeHereFull[FA]], 
                c = colorsMagStr[includeHereFull[FA]],
                s = 8, edgecolors = 'k', linewidths = .4)
    ax[i].set_ylim((-stressLim, stressLim))
    
    cellLength = np.mean([mod.dipLengthL[0][0], mod.strikeLengthL[0][0] ])
    
    for j in range(5):
        ax[i].plot([cellLength * j, cellLength * j],
                 [-stressLim, stressLim], c = 'black', linewidth = 1)
        
    ax[i].plot([np.amin(distPlot), np.amax(distPlot)], [0, 0], c = 'black', linewidth = 1)
    
    ax[i].set_ylabel('Greater $\Delta CFF$ (MPa)')    
    ax[i].set_title(titles[i])
    
ax[i].set_xlabel('Shortest distance to mainshock (m)')

        
plt.show()

# Discussion and Conclusions

## 6.1 Complexities of the Mainshock Surface
We tested the quality of the interpolated 3D fault surface through several approaches. One approach was to find how well the interpolated fault surface matches the orientation of the earthquakes that it interpolates (Figure 6). There are several aspects of this analysis to consider. When comparing the match of aftershock orientations to the mainshock models, there does not appear to be an improvement moving from the planar model to our 3D surface. This could have implications regarding the accuracy of our 3D fault surface, but it is also important that the L’Aquila mainshock ruptured on a relatively planar fault segment. It may be that this fault is planar enough that a 2D model is sufficient. Yet, this analysis is limited by inaccuracies from the focal mechanism inversion as well as aftershock locations. Faults with more curvature, such as the Campotosto fault, will likely show more certain improvement by incorporating 3D morphology. 

While ideally the aftershock orientations should have matched the 3D fault surface model well, when comparing the mismatch with each aftershock weighted equally, the aftershocks only mildly agree with any of the fault models (Figure 6). However, by weighting this mismatch with the energy of aftershocks $\left( \sim 10^M \right)$, the match between aftershocks and the 3D surface increases. This suggests that larger earthquakes are more closely aligned with the surface and/or the focal mechanisms of smaller events are less well identified. Oddly, a spatial pattern for agreement or disagreement between the focal mechanisms and fault surface does not appear to exist (Figure 6a), suggesting that disagreement is primarily from focal mechanism inversion error or from aftershocks truly rupturing at angles that are inconsistent with our inverted fault surface. 

The misalignment of small aftershocks with the main fault might be due to several reasons. One clear possibility is that the inversion of the orientation of low magnitude events might be less constrained than in the case of higher magnitude events. Another reason for such misalignment is that some aftershocks may be associated with conjugate faults instead of the main fault surface due to imperfect clustering (Figure 5b). There may be more unnoticed, smaller conjugate faults as well. These conjugate faults are responsible for part of the focal mechanism orientation misalignment as well as some positional mismatch between hypocenters and our 3D surface. Because these conjugate faults do not appear to constitute a major portion of the L’Aquila fault cluster, it appears that this is not a major problem. However, improved clustering should be done in the future to avoid this problem. 

However, this focal mechanism to fault model misalignment may have implications for the structure of the fault. Faults can become very sophisticated with damage zones at various scales and patterns (Peacock and Sanderson, 1991), and aftershock clusters tend to occur at the most complex regions of a fault surface where damage zones are the most extreme (Sibson 1989; Kim and Sanderson, 2008). Not only damage zones, but also the complexity of fault connectivity provides a means for aftershocks to rupture in directions and locations not in agreement with the main fault surface. One model for fault growth requires several smaller faults to link together, for instance through the breaching of relay ramps, in order to form larger faults (Childs et al., 2009). This results in complex rupture geometries. Relay ramps allow for a fault to be connected by many discontinuous faults at small and large scales. Relays have indeed been observed on the ground in the Paganica fault (Roberts et al., 2010). Faults can also be connected by extensional steps and contractional steps (Kim, peacock, and Sanderson, 2004). The tips of a fault can be particularly complex as well. Where fault displacement dies out at the tips of faults, displacement tends to end in splays (Kim, Peacock, and Sanderson, 2004). Splaying has been observed in the Paganica fault geologically (Cinti et al., 2011) and seismically (Chiaraluce et al., 2011). Given these issues, we are actually modelling a complex fault zone as a single 3D surface. Thus, we expect discontinuous sections of the fault zone to contain aftershocks which align poorly to the main fault surface. 

Given that we model the fault as simple and continuous 2D or 3D surfaces, we cannot model stresses induced by the fine scale fault zone characteristics discussed above. In general, these characteristics are probably too small to be detected. This poses a problem as such fault complexities can cause local stresses that greatly deviate from those modelled by smooth fault surfaces. Thus, using modern fault models to calculate ∆CFF, aftershocks associated stress perturbations originating from fault morphology discontinuities will appear erroneous. This, in part, is likely to explain why many events close to the mainshock are modelled to have negative ∆CFF while events far from the mainshock tend to have positive ∆CFF (Figures 9, 10, 11, and 13).

## 6.2 Slip Distribution Inversion and GPS Fit

Fit between modelled and observed co-seismic GPS displacements for our 2D and 3D GPS based mainshock models is very similar (Figure 7). Moving from the 2D to 3D model, horizontal displacement fit worsens while vertical fit improves (Table 2). The joint model finds the poorest fit to GPS data, though it should be much more realistic than at least our 2D GPS based model. This illustrates several problems. Our exclusion of a three-dimensional crustal model could be important. Also, GPS measurement errors can be very important and are often comparable in magnitude to measured displacement. This contributes to the important problem that slip distribution inversion results are non-unique. That is, for any model that reproduces the data decently, there are many different models that can also sufficiently reproduce the data. To help choose an appropriate model, complimentary data to the original dataset should be incorporated. The joint inversion included strong motion which is very complimentary to co-seismic geodetic data. Including strong motion and InSAR data within the joint inversion decreased the model’s ability to fit GPS data by ~15% (Cirella et al., 2012), yet ultimately resulted in a more realistic model. By using aftershock distribution to find fault morphology as we have done, information that is not present in co-seismic data is incorporated into the inversion. Incorporating other complementary data in our fault models would help to reduce the non-unique problem and extract realistic models which could further illuminate the roll of fault morphology in controlling stress and displacement.

## 6.3 Coulomb Stress

Generally, the joint inversion (Cirella et al. 2012) finds the highest number of events with positive ∆CFF. This suggests that incorporating the most realistic slip distribution by using complementary data sets is crucial. When moving from our GPS based planar inversion to our 3D inversion, we find a decrease in the number of actual focal mechanisms predicting positive ∆CFF (Figures 9 and 11). In part, this is likely a consequence of (i) the problem of obtaining the correct slip distribution from non-unique GPS based inversions, (ii) the assumed homogeneity of the crustal model, and (iii) lack of knowledge of the discontinuous 3D nature of the fault. The complex geometric relationship between aftershocks and a mainshock slip surface results in some error in the 3D rupture surface location and thus stress calculations. Future work could explore this effect. Although the planar GPS based mainshock model finds more focal mechanisms with +∆CFF, the 3D GPS based model finds more OOPs with +∆CFF. The OOP analysis has ~15 times the hypocenters and thus samples the spatial distribution of ∆CFF more thoroughly. In other words, the 3D model seems to better match the spatial distribution of aftershocks, while our planar model better matches the orientation of aftershocks. 

We note the correlation between slip distributions and aftershocks (Figure 10). As a general phenomenon, aftershocks are concentrated where slip distributions taper off or in portions of a fault that remain locked after an earthquake (Das and Henry, 2003). Stress tends to concentrate where slip distributions taper off, triggering aftershocks which relieve this stress. The high-resolution slip distribution of the joint inversion shows the strongest correlation between aftershock positions and slip distribution, though the more generalized and smooth slip distributions of the GPS based models still shows some correlation. Although aftershocks tend to concentrate where slip tapers off, ∆CFF results turn out negative most often in these regions particularly for our GPS based mainshock models. However, stress clearly must concentrate in these regions in order for so much seismicity to occur. This illustrates the unfortunate difficulty in modelling stresses most near the mainshock where aftershocks are most likely to occur.

The relationship between timing of aftershocks, their magnitudes, and their mainshock induced ∆CFF is shown in Figure 12a. Immediately after the mainshock, only for energy weighted aftershocks one observes a peak of positive ∆CFF, as seen by the increase in proportion of positive ∆CFF events when $10^M$ is used as a weight. Predicted ∆CFF decreases through time approximately until the MW 4.4 Campotosto earthquake in June. This is consistent with the idea that ∆CFF has the most control on aftershocks that follow shortly after the mainshock, and the ∆CFF from the mainshock decays and becomes less relevant with time. Yet, the trend becomes less clear close to and after August when ∆CFF is generally high again. When we exclude events which were not part of the L’Aquila cluster (Figure 12b), the temporal pattern for ∆CFF becomes very complicated. This is another illustration that stress transfer modelling is less reliable over short distances. 

Mechanisms other than static stress transfer are important in controlling seismic sequences as well (e.g. Freed, 2005 and references therein). Some examples include dynamic stress from seismic waves, fluid migration and pore pressure changes, and temporal variation in rock properties. These other mechanisms account for some of the aftershocks which according to mainshock models experienced negative ∆CFF from the mainshock. In some sense, ∆CFF should explain aftershocks better when they do not occur in proximity to the mainshock surface. This would be expected by the consideration that a brittle event is expected to weaken a fault, so it is not necessary to expect an increase in ∆CFF to trigger an aftershock very nearby the mainshock. We tested this hypothesis by calculating the percentage of events with positive ∆CFF, excluding the aftershocks that were part of the main fault surface cluster (Figure 11b). With this assumption, every model predicts a greater number of events with positive ∆CFF. Other studies have suggested that the role of fluid movement in driving aftershocks is important (Miller et al., 2004), particularly for the L’Aquila seismic sequence (Di Luccio et al., 2010; Lucente et al., 2010; Malagnini et al., 2012). Our results are consistent with this hypothesis because the intrusion of high-pressure fluids could have induced rupture where the mainshock did not generate positive ∆CFF. 

However, we have observed that ∆CFF is more sensitive to complexities in fault models at locations near the rupture (Figure 9). If ∆CFF is very sensitive to fault model parameters at locations very near the fault, then ∆CFF modelling becomes more error prone near the fault. Not only is the data used for inversions too low of resolution to reliably model stresses very near the mainshock, but discontinuities in the Okada and triangular dislocation functions make them unreliable for nearby events. In general, the distance between a fault patch and the point it calculates stress or displacement on should be greater than the size of the fault patch. This presents a particularly challenging problem in regard to our 3D surface. Because it is built to fit the location of aftershocks, the aftershocks are necessarily very close to the surface. Figure 13 shows that many aftershocks are excessively close to each fault model, resulting in unreliable stress results. The aftershocks in the L’Aquila cluster are almost all within a cell length from the 3D fault surface. This is undoubtedly one reason that the 3D surface finds a small number of aftershocks with +∆CFF. Note that the cell sizes are already impractically small, and to make the cells small enough that most points lie more than one cell size away from the fault would take an extremely fine triangular mesh. 

Given also the errors in mainshock and aftershock location, rupture models, and the lack of a 3D elastic model, it is most practical to use ∆CFF modelling at great distances. This is problematic as other large earthquakes, particularly on the Campotosto fault, likely had important control on such distant aftershocks. The mainshock should have had the most control on earthquakes that occurred on the L’Aquila fault where modelling is not fine enough to produce reliable ∆CFF calculations. This illustrates the importance of improving stress transfer models.

## 6.4 Limitations and Future Directions

The relationship between hypocenters and faults is complex, so extrapolating fault configuration and morphology through clustering presents several challenges: (i) some faults, especially conjugate faults, intersect each other and thus some hypocenters can ambiguously belong to either fault, (ii) several seismic events are located outside any fault, either due to earthquakes occurring far from a primary fault plane or due to the aftershocks being erroneously located, and (iii) hypocenters that belong to the same fault may be separated by large areas where there were no aftershocks, for instance where the mainshock relieved much stress (Wetzler et al., 2018). 

To overcome these problems, clustering was done in multiple iterations with the first iteration finding general clusters and the second iteration finding clusters in an appropriately spatially modified axis system. However, this does not completely solve the clustering issues. For instance, the technique of stretching hypocenters away from the best fit plane of a cluster would work poorly on a listric fault where the fault surface is oriented much differently than the best fit plane. A new or modified algorithm could be better suited. For instance, the k-planes algorithm (Bradley and Mangasarian, 2000) works by finding clusters based on each point’s least square distance from planes. By incorporating a way for planes to bend or to connect to each other appropriately, it will be possible to create a more effective algorithm for fault clustering. The method used to generate the 3D surface from hypocenters can also be improved upon. We used a grid of 7x7 knots, but by applying some smoothing function (analogous to how our slip distribution was smoothed), a finer grid of knots could be used. This could reduce the problem of strange spline behavior in locations where hypocenter distribution was greatly heterogeneous or sparse. Particularly with improvements to the method, obtained fault morphologies can be used for many things besides ∆CFF modelling such as modelling fluid flow, seismic waves, or understanding slip on low angle normal faults (such as in the Amatrice sequence). 

The location of maximum slip does not necessarily coincide with the hypocenter for any earthquake. This can be seen for instance in each of the slip distributions we have used (Figure 8). It may not be most meaningful to find ∆CFF at the hypocenter of aftershocks as is done here as well as in most other studies (King, Stein, and Lin, 1994; Nostro et al., 2005, Freed, 2005 and references therein). Rather, ∆CFF could be calculated at several locations on the fault surface that an earthquake might occur on, similar to Walters et al. (2009). Possibly, the distribution of this ∆CFF can give a better estimate of the slip distribution of the subsequent series of earthquakes. Using fault surface imaging such as done here offers the option to find induced ∆CFF over an entire rupture area of a fault (which can be many km long).

Some model-based limitations include the exclusion of topography, geological heterogeneities, fluid interaction, and the modification of rock properties as the sequence developed. Some of these limitations suggest considering the range of possible values of ∆CFF on each event. Whether aftershocks experienced positive or negative ∆CFF can be very sensitive to model parameters, so understanding the error margins of ∆CFF on aftershocks would help to better understand whether they truly experienced negative or positive ∆CFF. Statistical inversion of slip distributions can allow for propagation of slip error into calculated slip and stress (Yabuki and Matsu'ura, 1992; Cambiotti et al., 2017). Because stress can be represented as a linear function of slip on a fault patch, slip distribution error could be propagated to stress to give a probability that ∆CFF is positive for any event.

Many of the error inducing assumptions of this model can be mitigated using more sophisticated numerical modelling (Trasatti, Kyriakopoulos, and Chini, 2011, inverted for the source model of the L’Aquila mainshock using a finite element model, but the mainshock was assumed to lie on a plane). Using numerical models, one could incorporate topography, 3D mechanical heterogeneities, and potentially synthesize temporally variable fluid presence with static stress modelling. Combining a numerical approach with co-seismic data other than GPS which are available for the L’Aquila sequence, such as InSAR and strong motion, within this more general framework on a curved fault surface would allow for a very complete rupture model. This could be used to gain further insight into the roles of various mechanisms involved in the earthquake source process. 

Future work might involve simulating the entire time dependent evolution of the faulting surface. Previous work has suggested that seismicity moved towards north through time. Therefore, by reconstructing the surface morphology with partial sets of hypocenters, for example every week, this might give visual and quantitative information on the propagation of fluids in the crust and within the fault or other important insights such as the evolution of the 3D stress field in time.

A primary inspiration for this work is that when these methods are more fully developed, they can then be applied to more complicated earthquake sequences. In particular, the 2016/2017 Amatrice/Visso/Norcia sequence is an important target (Chiaraluce et al., 2017). The MW 6.5 event was preceded two months prior by a MW 6.0 event, and again only four days prior by a MW 5.9 event. Given the spatiotemporal correlation of these deadly events, they could provide a crucial view into earthquake triggering and hazard assessment. However, relocated seismicity has not provided a clear view of the associated faults, so extracting fault morphology in this sequence would facilitate a great deal of research. For instance, low angle normal fault seismicity is likely present in the aftershocks. Techniques such as the one developed here could reconstruct the low angle normal fault so that it can be studied in the context of triggering. This could greatly contribute to understanding how enigmatic low angle normal fault seismicity can occur. Further, the Amatrice sequence reactivated the Campotosto fault. This sequence and the L’Aquila sequence are thus intertwined, and it would be valuable to understand how the L’Aquila sequence contributed to the development of the Amatrice sequence.

# References
Please see main text.